In [1]:
import time
import re
from pathlib import Path
import pandas as pd
import numpy as np
import mojimoji
import datetime
import requests
from bs4 import BeautifulSoup
from tqdm import tqdm
import lxml

pd.set_option('display.max_columns', None)

In [2]:
read_dir_path = Path('/Users/masudaniwabinari/Desktop/hpb_scraping/data/データフレームの前処理でエラー')
read_all_file_path = read_dir_path.glob('*.csv')
_list = []
for read_file_path in read_all_file_path:
    _df = pd.read_csv(read_file_path)
    _list.append(_df)

df = pd.concat(objs=_list, axis=0).reset_index(drop=True)

/var/folders/0z/xm5_d7p905sdqddy7wnxk8jm0000gn/T/ipykernel_6663/2204518203.py:5: DtypeWarning: Columns (23,24,25,26,27) have mixed types. Specify dtype option on import or set low_memory=False.
  _df = pd.read_csv(read_file_path)
/var/folders/0z/xm5_d7p905sdqddy7wnxk8jm0000gn/T/ipykernel_6663/2204518203.py:5: DtypeWarning: Columns (23,24,25,26,27) have mixed types. Specify dtype option on import or set low_memory=False.
  _df = pd.read_csv(read_file_path)


In [3]:
df

,県,エリア,サロン名,URL,住所,アクセス,カット料金,セット面の数,ブログ投稿数,口コミ数,スタッフ数,口コミ数1番人気クーポン名,1番人気クーポン価格,2番人気クーポン名,2番人気クーポン価格,3番人気クーポン名,3番人気クーポン価格,初回予約金額,2回目以降予約金額,クーポン数,メニュー数,スタイル数,ブログ数,口コミスコアサロン全平均,雰囲気の全平均,接客サービスの全平均,技術・仕上がりの全平均,メニュー。料金の全平均,名前,性別、年齢、属性,投稿日時,総合,雰囲気,接客サービス,技術・仕上がり,メニュー・料金,口コミ本文,選択されたクーポン,メニューの種類,コメントへの返信
0,徳島,北島・藍住・松茂・鳴門の美容院・美容室・ヘアサロン,HIBIKA,https://beauty.hotpepper.jp/slnH000243953/?cstt=1,徳島県板野郡北島町中村字長池32-2,北島郵便局前、北島小学校隣,"¥4,400～",セット面6席,1668件 UP,1459件 UP,6.0,【ご新規様限定！HIBIKA10周年記念】フルカラー＋カット＋トリートメント,"¥10,340",１０周年記念千田クーポン【春夏カラーとリラックス髪質改善トリートメント】,"¥13,000",10周年記念西山クーポン【カラーとスパの贅沢リラクゼーションコース】,"¥17,160","¥10,000～¥10,999","¥10,000～¥10,999",43.0,40,532.0,1668.0,4.84,4.8,4.9,4.9,4.5,あさみさん,（女性/30代前半）,[投稿日] 2023/5/20,5.0,4.0,5.0,5.0,4.0,今回もトリートメントでツヤツヤになり\n大満足です！ありがとうございました。♪\n綺麗にして...,千田クーポン【サロンでしか味わえない手触りに・・・。ハイダメージもOK】,[施術メニュー] トリートメント,無
1,徳島,北島・藍住・松茂・鳴門の美容院・美容室・ヘアサロン,HIBIKA,https://beauty.hotpepper.jp/slnH000243953/?cstt=1,徳島県板野郡北島町中村字長池32-2,北島郵便局前、北島小学校隣,"¥4,400～",セット面6席,1668件 UP,1459件 UP,6.0,【ご新規様限定！HIBIKA10周年記念】フルカラー＋カット＋トリートメント,"¥10,340",１０周年記念千田クーポン【春夏カラーとリラックス髪質改善トリートメント】,"¥13,000",10周年記念西山クーポン【カラーとスパの贅沢リラクゼーションコース】,"¥17,160","¥10,000～¥10,999","¥10,000～¥10,999",43.0,40,532.0,1668.0,4.84,4.8,4.9,4.9,4.5,すずくまさん,（女性/40代/パート・アルバイト）,[投稿日] 2023/5/20,5.0,5.0,5.0,5.0,5.0,いつもこちらの希望をよく聞いていただけ、仕上がりの感じもとても良いので安心してお任せできます...,クーポン利用なし,[施術メニュー] カット,無
2,徳島,北島・藍住・松茂・鳴門の美容院・美容室・ヘアサロン,HIBIKA,https://beauty.hotpepper.jp/slnH000243953/?cstt=1,徳島県板野郡北島町中村字長池32-2,北島郵便局前、北島小学校隣,"¥4,400～",セット面6席,1668件 UP,1459件 UP,6.0,【ご新規様限定！HIBIKA10周年記念】フルカラー＋カット＋トリートメント,"¥10,340",１０周年記念千田クーポン【春夏カラーとリラックス髪質改善トリートメント】,"¥13,000",10周年記念西山クーポン【カラーとスパの贅沢リラクゼーションコース】,"¥17,160","¥10,000～¥10,999","¥10,000～¥10,999",43.0,40,532.0,1668.0,4.84,4.8,4.9,4.9,4.5,gonさん,（女性/20代前半/学生）,[投稿日] 2023/5/19,5.0,5.0,5.0,5.0,5.0,カラーをお願いしました。\n髪色の相談を親身になって聞いてくださり、可愛く仕上げてもらうこと...,クーポン利用なし,[施術メニュー] カラー,無
3,徳島,北島・藍住・松茂・鳴門の美容院・美容室・ヘアサロン,HIBIKA,https://beauty.hotpepper.jp/slnH000243953/?cstt=1,徳島県板野郡北島町中村字長池32-2,北島郵便局前、北島小学校隣,"¥4,400～",セット面6席,1668件 UP,1459件 UP,6.0,【ご新規様限定！HIBIKA10周年記念】フルカラー＋カット＋トリートメント,"¥10,340",１０周年記念千田クーポン【春夏カラーとリラックス髪質改善トリートメント】,"¥13,000",10周年記念西山クーポン【カラーとスパの贅沢リラクゼーションコース】,"¥17,160","¥10,000～¥10,999","¥10,000～¥10,999",43.0,40,532.0,1668.0,4.84,4.8,4.9,4.9,4.5,はるさん,（女性/40代）,[投稿日] 2023/5/19,4.0,4.0,5.0,5.0,3.0,今回も帰ってからの痒み等の不調はありません。\n軽くなったので、シャンプーや乾かすのが、楽に...,10周年記念坂田クーポン【むくみコリ疲れ、排出☆オールハンドすっきりスパ】,[施術メニュー] カット、ヘッドスパ、その他,有
4,徳島,北島・藍住・松茂・鳴門の美容院・美容室・ヘアサロン,HIBIKA,https://beauty.hotpepper.jp/slnH000243953/?cstt=1,徳島県板野郡北島町中村字長池32-2,北島郵便局前、北島小学校隣,"¥4,400～",セット面6席,1668件 UP,1459件 UP,6.0,【ご新規様限定！HIBIKA10周年記念】フルカラー＋カット＋トリートメント,"¥10,340",１０周年記念千田クーポン【春夏カラーとリラックス髪質改善トリートメント】,"¥13,000",10周年記念西山クーポン【カラーとスパの贅沢リラクゼーションコース】,"¥17,160","¥10,000～¥10,999","¥10,000～¥10,999",43.0,40,532.0,1668.0,4.84,4.8,4.9,4.9,4.5,やまさん,（女性/20代後半/会社員）,[投稿日] 2023/5/18,4.0,4.0,4.0,4.0,3.0,以前担当スタイリストさんに施術いただいた仕上がりがとても良く、2回目もお願いしました。カウン...,【髪の広がりとカラーのダメージが気になる方へ】,[施術メニュー] カット、カラー、トリートメント,無
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
223233,福山・尾道,福山駅前・三吉周辺の美容院・美容室・ヘアサロン,Chou Chou Hair 【シュシュヘアー】,https://beauty.hotpepper.jp/slnH000400505/?cst...,広島県福山市木之庄町３-3-20,福山駅から徒歩15分【ファミリーマート木之庄店から徒歩1分】【髪質改善/縮毛矯正】,"¥5,400",セット面2席,NaN,12件,1.0,NaN,NaN,NaN,NaN,NaN,NaN,"¥23,000～¥23,999",データ収集中,NaN,10,NaN,NaN,4.75,4.8,5.0,5.0,4.5,じゃむさん,（女性/10代後半）,[投稿日] 2022/1/5,5.0,5.0,5.0,5.0,5.0,今日はありがとうございました！\n髪の毛のケアについて色々と丁寧に教えてもらい、家でもケアを...,クーポン利用なし,[施術メニュー] トリートメント、その他,有
223234,福山・尾道,福山駅前・三吉周辺の美容院・美容室・ヘアサロン,Chou Chou Hair 【シュシュヘアー】,https://beauty.hotpepper.jp/slnH000400505/?cst...,広島県福山市木之庄町３-3-20,福山駅から徒歩15分【ファミリーマート木之庄店から徒歩1分】【髪質改善/縮毛矯正】,"¥5,400",セット面2席,NaN,12件,1.0,NaN,NaN,NaN,NaN,NaN,NaN,"¥23,000～¥23,999",データ収集中,NaN,10,NaN,NaN,4.75,4.8,5.0,5.0,4.5,しぃちゃんさん,（女性/20代前半/学生）,[投稿日] 2021/11/15,5.0,5.0,5.0,5.0,4.0,初めての髪質改善だったけど、とても綺麗な仕上がりに驚きました。自分の髪とは信じれないほどサラ...,クーポン利用なし,[施術メニュー] カット、カラー、トリートメント,有
223235,福山・尾道,福山駅前・三吉周辺の美容院・美容室・ヘアサロン,Chou Chou Hair

In [4]:
df['エリア'] = df['エリア'].str.replace('の美容院・美容室・ヘアサロン', '')

In [5]:
df.head()

,県,エリア,サロン名,URL,住所,アクセス,カット料金,セット面の数,ブログ投稿数,口コミ数,スタッフ数,口コミ数1番人気クーポン名,1番人気クーポン価格,2番人気クーポン名,2番人気クーポン価格,3番人気クーポン名,3番人気クーポン価格,初回予約金額,2回目以降予約金額,クーポン数,メニュー数,スタイル数,ブログ数,口コミスコアサロン全平均,雰囲気の全平均,接客サービスの全平均,技術・仕上がりの全平均,メニュー。料金の全平均,名前,性別、年齢、属性,投稿日時,総合,雰囲気,接客サービス,技術・仕上がり,メニュー・料金,口コミ本文,選択されたクーポン,メニューの種類,コメントへの返信
0,徳島,北島・藍住・松茂・鳴門,HIBIKA,https://beauty.hotpepper.jp/slnH000243953/?cstt=1,徳島県板野郡北島町中村字長池32-2,北島郵便局前、北島小学校隣,"¥4,400～",セット面6席,1668件 UP,1459件 UP,6.0,【ご新規様限定！HIBIKA10周年記念】フルカラー＋カット＋トリートメント,"¥10,340",１０周年記念千田クーポン【春夏カラーとリラックス髪質改善トリートメント】,"¥13,000",10周年記念西山クーポン【カラーとスパの贅沢リラクゼーションコース】,"¥17,160","¥10,000～¥10,999","¥10,000～¥10,999",43.0,40,532.0,1668.0,4.84,4.8,4.9,4.9,4.5,あさみさん,（女性/30代前半）,[投稿日] 2023/5/20,5.0,4.0,5.0,5.0,4.0,今回もトリートメントでツヤツヤになり\n大満足です！ありがとうございました。♪\n綺麗にして...,千田クーポン【サロンでしか味わえない手触りに・・・。ハイダメージもOK】,[施術メニュー] トリートメント,無
1,徳島,北島・藍住・松茂・鳴門,HIBIKA,https://beauty.hotpepper.jp/slnH000243953/?cstt=1,徳島県板野郡北島町中村字長池32-2,北島郵便局前、北島小学校隣,"¥4,400～",セット面6席,1668件 UP,1459件 UP,6.0,【ご新規様限定！HIBIKA10周年記念】フルカラー＋カット＋トリートメント,"¥10,340",１０周年記念千田クーポン【春夏カラーとリラックス髪質改善トリートメント】,"¥13,000",10周年記念西山クーポン【カラーとスパの贅沢リラクゼーションコース】,"¥17,160","¥10,000～¥10,999","¥10,000～¥10,999",43.0,40,532.0,1668.0,4.84,4.8,4.9,4.9,4.5,すずくまさん,（女性/40代/パート・アルバイト）,[投稿日] 2023/5/20,5.0,5.0,5.0,5.0,5.0,いつもこちらの希望をよく聞いていただけ、仕上がりの感じもとても良いので安心してお任せできます...,クーポン利用なし,[施術メニュー] カット,無
2,徳島,北島・藍住・松茂・鳴門,HIBIKA,https://beauty.hotpepper.jp/slnH000243953/?cstt=1,徳島県板野郡北島町中村字長池32-2,北島郵便局前、北島小学校隣,"¥4,400～",セット面6席,1668件 UP,1459件 UP,6.0,【ご新規様限定！HIBIKA10周年記念】フルカラー＋カット＋トリートメント,"¥10,340",１０周年記念千田クーポン【春夏カラーとリラックス髪質改善トリートメント】,"¥13,000",10周年記念西山クーポン【カラーとスパの贅沢リラクゼーションコース】,"¥17,160","¥10,000～¥10,999","¥10,000～¥10,999",43.0,40,532.0,1668.0,4.84,4.8,4.9,4.9,4.5,gonさん,（女性/20代前半/学生）,[投稿日] 2023/5/19,5.0,5.0,5.0,5.0,5.0,カラーをお願いしました。\n髪色の相談を親身になって聞いてくださり、可愛く仕上げてもらうこと...,クーポン利用なし,[施術メニュー] カラー,無
3,徳島,北島・藍住・松茂・鳴門,HIBIKA,https://beauty.hotpepper.jp/slnH000243953/?cstt=1,徳島県板野郡北島町中村字長池32-2,北島郵便局前、北島小学校隣,"¥4,400～",セット面6席,1668件 UP,1459件 UP,6.0,【ご新規様限定！HIBIKA10周年記念】フルカラー＋カット＋トリートメント,"¥10,340",１０周年記念千田クーポン【春夏カラーとリラックス髪質改善トリートメント】,"¥13,000",10周年記念西山クーポン【カラーとスパの贅沢リラクゼーションコース】,"¥17,160","¥10,000～¥10,999","¥10,000～¥10,999",43.0,40,532.0,1668.0,4.84,4.8,4.9,4.9,4.5,はるさん,（女性/40代）,[投稿日] 2023/5/19,4.0,4.0,5.0,5.0,3.0,今回も帰ってからの痒み等の不調はありません。\n軽くなったので、シャンプーや乾かすのが、楽に...,10周年記念坂田クーポン【むくみコリ疲れ、排出☆オールハンドすっきりスパ】,[施術メニュー] カット、ヘッドスパ、その他,有
4,徳島,北島・藍住・松茂・鳴門,HIBIKA,https://beauty.hotpepper.jp/slnH000243953/?cstt=1,徳島県板野郡北島町中村字長池32-2,北島郵便局前、北島小学校隣,"¥4,400～",セット面6席,1668件 UP,1459件 UP,6.0,【ご新規様限定！HIBIKA10周年記念】フルカラー＋カット＋トリートメント,"¥10,340",１０周年記念千田クーポン【春夏カラーとリラックス髪質改善トリートメント】,"¥13,000",10周年記念西山クーポン【カラーとスパの贅沢リラクゼーションコース】,"¥17,160","¥10,000～¥10,999","¥10,000～¥10,999",43.0,40,532.0,1668.0,4.84,4.8,4.9,4.9,4.5,やまさん,（女性/20代後半/会社員）,[投稿日] 2023/5/18,4.0,4.0,4.0,4.0,3.0,以前担当スタイリストさんに施術いただいた仕上がりがとても良く、2回目もお願いしました。カウン...,【髪の広がりとカラーのダメージが気になる方へ】,[施術メニュー] カット、カラー、トリートメント,無


# 処理したカラム
- エリア

In [6]:
df['投稿日時'] = df['投稿日時'].str.replace('[投稿日] ', '', regex=False)
df['投稿日時']

0          2023/5/20
1          2023/5/20
2          2023/5/19
3          2023/5/19
4          2023/5/18
             ...    
223233      2022/1/5
223234    2021/11/15
223235     2021/11/5
223236     2021/8/19
223237      2021/7/3
Name: 投稿日時, Length: 223238, dtype: object

# 処理したカラム
- エリア
- 投稿日時

In [7]:
df['セット面の数'] = df['セット面の数'].str.replace('セット面', '').str.replace('席', '')
df['セット面の数']

0         6
1         6
2         6
3         6
4         6
         ..
223233    2
223234    2
223235    2
223236    2
223237    2
Name: セット面の数, Length: 223238, dtype: object

# 処理したカラム
- エリア
- 投稿日時
- セット面の数

In [8]:
df.head(1)

,県,エリア,サロン名,URL,住所,アクセス,カット料金,セット面の数,ブログ投稿数,口コミ数,スタッフ数,口コミ数1番人気クーポン名,1番人気クーポン価格,2番人気クーポン名,2番人気クーポン価格,3番人気クーポン名,3番人気クーポン価格,初回予約金額,2回目以降予約金額,クーポン数,メニュー数,スタイル数,ブログ数,口コミスコアサロン全平均,雰囲気の全平均,接客サービスの全平均,技術・仕上がりの全平均,メニュー。料金の全平均,名前,性別、年齢、属性,投稿日時,総合,雰囲気,接客サービス,技術・仕上がり,メニュー・料金,口コミ本文,選択されたクーポン,メニューの種類,コメントへの返信
0,徳島,北島・藍住・松茂・鳴門,HIBIKA,https://beauty.hotpepper.jp/slnH000243953/?cstt=1,徳島県板野郡北島町中村字長池32-2,北島郵便局前、北島小学校隣,"¥4,400～",6,1668件 UP,1459件 UP,6.0,【ご新規様限定！HIBIKA10周年記念】フルカラー＋カット＋トリートメント,"¥10,340",１０周年記念千田クーポン【春夏カラーとリラックス髪質改善トリートメント】,"¥13,000",10周年記念西山クーポン【カラーとスパの贅沢リラクゼーションコース】,"¥17,160","¥10,000～¥10,999","¥10,000～¥10,999",43.0,40,532.0,1668.0,4.84,4.8,4.9,4.9,4.5,あさみさん,（女性/30代前半）,2023/5/20,5.0,4.0,5.0,5.0,4.0,今回もトリートメントでツヤツヤになり\n大満足です！ありがとうございました。♪\n綺麗にして...,千田クーポン【サロンでしか味わえない手触りに・・・。ハイダメージもOK】,[施術メニュー] トリートメント,無


In [9]:
df['カット料金'] = df['カット料金'].str.replace('¥', '').str.replace(',', '').str.replace('～', '').str.replace('~', '')
df.head(1)

,県,エリア,サロン名,URL,住所,アクセス,カット料金,セット面の数,ブログ投稿数,口コミ数,スタッフ数,口コミ数1番人気クーポン名,1番人気クーポン価格,2番人気クーポン名,2番人気クーポン価格,3番人気クーポン名,3番人気クーポン価格,初回予約金額,2回目以降予約金額,クーポン数,メニュー数,スタイル数,ブログ数,口コミスコアサロン全平均,雰囲気の全平均,接客サービスの全平均,技術・仕上がりの全平均,メニュー。料金の全平均,名前,性別、年齢、属性,投稿日時,総合,雰囲気,接客サービス,技術・仕上がり,メニュー・料金,口コミ本文,選択されたクーポン,メニューの種類,コメントへの返信
0,徳島,北島・藍住・松茂・鳴門,HIBIKA,https://beauty.hotpepper.jp/slnH000243953/?cstt=1,徳島県板野郡北島町中村字長池32-2,北島郵便局前、北島小学校隣,4400,6,1668件 UP,1459件 UP,6.0,【ご新規様限定！HIBIKA10周年記念】フルカラー＋カット＋トリートメント,"¥10,340",１０周年記念千田クーポン【春夏カラーとリラックス髪質改善トリートメント】,"¥13,000",10周年記念西山クーポン【カラーとスパの贅沢リラクゼーションコース】,"¥17,160","¥10,000～¥10,999","¥10,000～¥10,999",43.0,40,532.0,1668.0,4.84,4.8,4.9,4.9,4.5,あさみさん,（女性/30代前半）,2023/5/20,5.0,4.0,5.0,5.0,4.0,今回もトリートメントでツヤツヤになり\n大満足です！ありがとうございました。♪\n綺麗にして...,千田クーポン【サロンでしか味わえない手触りに・・・。ハイダメージもOK】,[施術メニュー] トリートメント,無


# 処理したカラム
- エリア
- 投稿日時
- セット面の数
- カット料金

In [10]:
df['ブログ投稿数'] = df['ブログ投稿数'].str.replace('件', '').str.replace('　', '').str.replace(' ', '').str.replace('UP', '')
df['ブログ投稿数']

0         1668
1         1668
2         1668
3         1668
4         1668
          ... 
223233     NaN
223234     NaN
223235     NaN
223236     NaN
223237     NaN
Name: ブログ投稿数, Length: 223238, dtype: object

In [11]:
df['ブログ投稿数'] = df['ブログ投稿数'].fillna(0)
df.tail()

,県,エリア,サロン名,URL,住所,アクセス,カット料金,セット面の数,ブログ投稿数,口コミ数,スタッフ数,口コミ数1番人気クーポン名,1番人気クーポン価格,2番人気クーポン名,2番人気クーポン価格,3番人気クーポン名,3番人気クーポン価格,初回予約金額,2回目以降予約金額,クーポン数,メニュー数,スタイル数,ブログ数,口コミスコアサロン全平均,雰囲気の全平均,接客サービスの全平均,技術・仕上がりの全平均,メニュー。料金の全平均,名前,性別、年齢、属性,投稿日時,総合,雰囲気,接客サービス,技術・仕上がり,メニュー・料金,口コミ本文,選択されたクーポン,メニューの種類,コメントへの返信
223233,福山・尾道,福山駅前・三吉周辺,Chou Chou Hair 【シュシュヘアー】,https://beauty.hotpepper.jp/slnH000400505/?cst...,広島県福山市木之庄町３-3-20,福山駅から徒歩15分【ファミリーマート木之庄店から徒歩1分】【髪質改善/縮毛矯正】,5400,2,0,12件,1.0,NaN,NaN,NaN,NaN,NaN,NaN,"¥23,000～¥23,999",データ収集中,NaN,10,NaN,NaN,4.75,4.8,5.0,5.0,4.5,じゃむさん,（女性/10代後半）,2022/1/5,5.0,5.0,5.0,5.0,5.0,今日はありがとうございました！\n髪の毛のケアについて色々と丁寧に教えてもらい、家でもケアを...,クーポン利用なし,[施術メニュー] トリートメント、その他,有
223234,福山・尾道,福山駅前・三吉周辺,Chou Chou Hair 【シュシュヘアー】,https://beauty.hotpepper.jp/slnH000400505/?cst...,広島県福山市木之庄町３-3-20,福山駅から徒歩15分【ファミリーマート木之庄店から徒歩1分】【髪質改善/縮毛矯正】,5400,2,0,12件,1.0,NaN,NaN,NaN,NaN,NaN,NaN,"¥23,000～¥23,999",データ収集中,NaN,10,NaN,NaN,4.75,4.8,5.0,5.0,4.5,しぃちゃんさん,（女性/20代前半/学生）,2021/11/15,5.0,5.0,5.0,5.0,4.0,初めての髪質改善だったけど、とても綺麗な仕上がりに驚きました。自分の髪とは信じれないほどサラ...,クーポン利用なし,[施術メニュー] カット、カラー、トリートメント,有
223235,福山・尾道,福山駅前・三吉周辺,Chou Chou Hair 【シュシュヘアー】,https://beauty.hotpepper.jp/slnH000400505/?cst...,広島県福山市木之庄町３-3-20,福山駅から徒歩15分【ファミリーマート木之庄店から徒歩1分】【髪質改善/縮毛矯正】,5400,2,0,12件,1.0,NaN,NaN,NaN,NaN,NaN,NaN,"¥23,000～¥23,999",データ収集中,NaN,10,NaN,NaN,4.75,4.8,5.0,5.0,4.5,エマさん,（女性/20代前半/会社員）,2021/11/5,5.0,5.0,5.0,5.0,3.0,髪がとても綺麗になって感動しました!!\nお値段は高いけどそれ以上に効果がありました！\nぜ...,クーポン利用なし,[施術メニュー] カラー、トリートメント,有
223236,福山・尾道,福山駅前・三吉周辺,Chou Chou Hair 【シュシュヘアー】,https://beauty.hotpepper.jp/slnH000400505/?cst...,広島県福山市木之庄町３-3-20,福山駅から徒歩15分【ファミリーマート木之庄店から徒歩1分】【髪質改善/縮毛矯正】,5400,2,0,12件,1.0,NaN,NaN,NaN,NaN,NaN,NaN,"¥23,000～¥23,999",データ収集中,NaN,10,NaN,NaN,4.75,4.8,5.0,5.0,4.5,サラさん,（女性/10代後半/学生）,2021/8/19,5.0,5.0,5.0,5.0,5.0,初めての髪質改善だったけど最初の触り心地と終わってからの触り心地が全く違って驚きました。\n...,クーポン利用なし,[施術メニュー] トリートメント、その他,有
223237,福山・尾道,福山駅前・三吉周辺,Chou Chou Hair 【シュシュヘアー】,https://beauty.hotpepper.jp/slnH000400505/?cst...,広島県福山市木之庄町３-3-20,福山駅から徒歩15分【ファミリーマート木之庄店から徒歩1分】【髪質改善/縮毛矯正】,5400,2,0,12件,1.0,NaN,NaN,NaN,NaN,NaN,NaN,"¥23,000～¥23,999",データ収集中,NaN,10,NaN,NaN,4.75,4.8,5.0,5.0,4.5,きいさん,（女性/20代前半/学生）,2021/7/3,5.0,5.0,5.0,5.0,4.0,初めての髪質改善を行って貰いました。\n最初にどこが傷んでいるかなど詳しくお話をしていただき...,クーポン利用なし,[施術メニュー] トリートメント、その他,有


# 処理したカラム
- エリア
- 投稿日時
- セット面の数
- カット料金
- ブログ投稿数

In [12]:
df['口コミ数'] = df['口コミ数'].str.replace('件', '').str.replace('　', '').str.replace(' ', '').str.replace('UP', '')
df['口コミ数']

0         1459
1         1459
2         1459
3         1459
4         1459
          ... 
223233      12
223234      12
223235      12
223236      12
223237      12
Name: 口コミ数, Length: 223238, dtype: object

In [13]:
df['口コミ数'] = df['口コミ数'].fillna(0)
df.tail(1)

,県,エリア,サロン名,URL,住所,アクセス,カット料金,セット面の数,ブログ投稿数,口コミ数,スタッフ数,口コミ数1番人気クーポン名,1番人気クーポン価格,2番人気クーポン名,2番人気クーポン価格,3番人気クーポン名,3番人気クーポン価格,初回予約金額,2回目以降予約金額,クーポン数,メニュー数,スタイル数,ブログ数,口コミスコアサロン全平均,雰囲気の全平均,接客サービスの全平均,技術・仕上がりの全平均,メニュー。料金の全平均,名前,性別、年齢、属性,投稿日時,総合,雰囲気,接客サービス,技術・仕上がり,メニュー・料金,口コミ本文,選択されたクーポン,メニューの種類,コメントへの返信
223237,福山・尾道,福山駅前・三吉周辺,Chou Chou Hair 【シュシュヘアー】,https://beauty.hotpepper.jp/slnH000400505/?cst...,広島県福山市木之庄町３-3-20,福山駅から徒歩15分【ファミリーマート木之庄店から徒歩1分】【髪質改善/縮毛矯正】,5400,2,0,12,1.0,NaN,NaN,NaN,NaN,NaN,NaN,"¥23,000～¥23,999",データ収集中,NaN,10,NaN,NaN,4.75,4.8,5.0,5.0,4.5,きいさん,（女性/20代前半/学生）,2021/7/3,5.0,5.0,5.0,5.0,4.0,初めての髪質改善を行って貰いました。\n最初にどこが傷んでいるかなど詳しくお話をしていただき...,クーポン利用なし,[施術メニュー] トリートメント、その他,有


# 処理したカラム
- エリア
- 投稿日時
- セット面の数
- カット料金
- ブログ投稿数
- 口コミ数

In [14]:
df.rename(columns={'口コミ数1番人気クーポン名':'1番人気クーポン名'}, inplace=True)

In [15]:
df.tail(1)

,県,エリア,サロン名,URL,住所,アクセス,カット料金,セット面の数,ブログ投稿数,口コミ数,スタッフ数,1番人気クーポン名,1番人気クーポン価格,2番人気クーポン名,2番人気クーポン価格,3番人気クーポン名,3番人気クーポン価格,初回予約金額,2回目以降予約金額,クーポン数,メニュー数,スタイル数,ブログ数,口コミスコアサロン全平均,雰囲気の全平均,接客サービスの全平均,技術・仕上がりの全平均,メニュー。料金の全平均,名前,性別、年齢、属性,投稿日時,総合,雰囲気,接客サービス,技術・仕上がり,メニュー・料金,口コミ本文,選択されたクーポン,メニューの種類,コメントへの返信
223237,福山・尾道,福山駅前・三吉周辺,Chou Chou Hair 【シュシュヘアー】,https://beauty.hotpepper.jp/slnH000400505/?cst...,広島県福山市木之庄町３-3-20,福山駅から徒歩15分【ファミリーマート木之庄店から徒歩1分】【髪質改善/縮毛矯正】,5400,2,0,12,1.0,NaN,NaN,NaN,NaN,NaN,NaN,"¥23,000～¥23,999",データ収集中,NaN,10,NaN,NaN,4.75,4.8,5.0,5.0,4.5,きいさん,（女性/20代前半/学生）,2021/7/3,5.0,5.0,5.0,5.0,4.0,初めての髪質改善を行って貰いました。\n最初にどこが傷んでいるかなど詳しくお話をしていただき...,クーポン利用なし,[施術メニュー] トリートメント、その他,有


In [16]:
df[['1番人気クーポン名',
'2番人気クーポン名',
'3番人気クーポン名']] =\
df[['1番人気クーポン名',
'2番人気クーポン名',
'3番人気クーポン名']].fillna('')
df.tail(1)

,県,エリア,サロン名,URL,住所,アクセス,カット料金,セット面の数,ブログ投稿数,口コミ数,スタッフ数,1番人気クーポン名,1番人気クーポン価格,2番人気クーポン名,2番人気クーポン価格,3番人気クーポン名,3番人気クーポン価格,初回予約金額,2回目以降予約金額,クーポン数,メニュー数,スタイル数,ブログ数,口コミスコアサロン全平均,雰囲気の全平均,接客サービスの全平均,技術・仕上がりの全平均,メニュー。料金の全平均,名前,性別、年齢、属性,投稿日時,総合,雰囲気,接客サービス,技術・仕上がり,メニュー・料金,口コミ本文,選択されたクーポン,メニューの種類,コメントへの返信
223237,福山・尾道,福山駅前・三吉周辺,Chou Chou Hair 【シュシュヘアー】,https://beauty.hotpepper.jp/slnH000400505/?cst...,広島県福山市木之庄町３-3-20,福山駅から徒歩15分【ファミリーマート木之庄店から徒歩1分】【髪質改善/縮毛矯正】,5400,2,0,12,1.0,,NaN,,NaN,,NaN,"¥23,000～¥23,999",データ収集中,NaN,10,NaN,NaN,4.75,4.8,5.0,5.0,4.5,きいさん,（女性/20代前半/学生）,2021/7/3,5.0,5.0,5.0,5.0,4.0,初めての髪質改善を行って貰いました。\n最初にどこが傷んでいるかなど詳しくお話をしていただき...,クーポン利用なし,[施術メニュー] トリートメント、その他,有


In [17]:
df['1番人気クーポン価格'] = df['1番人気クーポン価格'].str.replace('¥','').str.replace(',', '')
df['2番人気クーポン価格'] = df['2番人気クーポン価格'].str.replace('¥','').str.replace(',', '')
df['3番人気クーポン価格'] = df['3番人気クーポン価格'].str.replace('¥','').str.replace(',', '')

df[['1番人気クーポン価格',
'2番人気クーポン価格',
'3番人気クーポン価格']] =\
df[['1番人気クーポン価格',
'2番人気クーポン価格',
'3番人気クーポン価格']].fillna(0)
df.tail(1)

,県,エリア,サロン名,URL,住所,アクセス,カット料金,セット面の数,ブログ投稿数,口コミ数,スタッフ数,1番人気クーポン名,1番人気クーポン価格,2番人気クーポン名,2番人気クーポン価格,3番人気クーポン名,3番人気クーポン価格,初回予約金額,2回目以降予約金額,クーポン数,メニュー数,スタイル数,ブログ数,口コミスコアサロン全平均,雰囲気の全平均,接客サービスの全平均,技術・仕上がりの全平均,メニュー。料金の全平均,名前,性別、年齢、属性,投稿日時,総合,雰囲気,接客サービス,技術・仕上がり,メニュー・料金,口コミ本文,選択されたクーポン,メニューの種類,コメントへの返信
223237,福山・尾道,福山駅前・三吉周辺,Chou Chou Hair 【シュシュヘアー】,https://beauty.hotpepper.jp/slnH000400505/?cst...,広島県福山市木之庄町３-3-20,福山駅から徒歩15分【ファミリーマート木之庄店から徒歩1分】【髪質改善/縮毛矯正】,5400,2,0,12,1.0,,0,,0,,0,"¥23,000～¥23,999",データ収集中,NaN,10,NaN,NaN,4.75,4.8,5.0,5.0,4.5,きいさん,（女性/20代前半/学生）,2021/7/3,5.0,5.0,5.0,5.0,4.0,初めての髪質改善を行って貰いました。\n最初にどこが傷んでいるかなど詳しくお話をしていただき...,クーポン利用なし,[施術メニュー] トリートメント、その他,有


In [18]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 223238 entries, 0 to 223237
Data columns (total 40 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   県             223238 non-null  object 
 1   エリア           223238 non-null  object 
 2   サロン名          223238 non-null  object 
 3   URL           223238 non-null  object 
 4   住所            223238 non-null  object 
 5   アクセス          223238 non-null  object 
 6   カット料金         223238 non-null  object 
 7   セット面の数        223238 non-null  object 
 8   ブログ投稿数        223238 non-null  object 
 9   口コミ数          223238 non-null  object 
 10  スタッフ数         223189 non-null  float64
 11  1番人気クーポン名     223238 non-null  object 
 12  1番人気クーポン価格    223238 non-null  object 
 13  2番人気クーポン名     223238 non-null  object 
 14  2番人気クーポン価格    223238 non-null  object 
 15  3番人気クーポン名     223238 non-null  object 
 16  3番人気クーポン価格    223238 non-null  object 
 17  初回予約金額        223053 non-null  object 
 18  2回目以

# 処理したカラム
- エリア
- 投稿日時
- セット面の数
- カット料金
- ブログ投稿数
- 口コミ数
- 1〜3番人気クーポン名、価格

In [19]:
df['初回予約金額'].fillna(0, inplace=True)
df['初回予約金額'].isna().sum()

0

In [20]:
df = df.reset_index(drop=True)

In [21]:
_list_for_df = []
for _list in df['初回予約金額'].str.replace('¥', '').str.replace(',', '').str.split('～'):
    if  _list is np.nan or len(_list) < 2:
        _list=['0', '0']
    _list_for_df.append(_list)

In [22]:
df = pd.concat([df, pd.DataFrame(_list_for_df, columns=['初回予約金額最小', '初回予約金額最大'])], axis=1)

In [23]:
df.drop(['初回予約金額'], axis=1, inplace=True)

In [24]:
df

,県,エリア,サロン名,URL,住所,アクセス,カット料金,セット面の数,ブログ投稿数,口コミ数,スタッフ数,1番人気クーポン名,1番人気クーポン価格,2番人気クーポン名,2番人気クーポン価格,3番人気クーポン名,3番人気クーポン価格,2回目以降予約金額,クーポン数,メニュー数,スタイル数,ブログ数,口コミスコアサロン全平均,雰囲気の全平均,接客サービスの全平均,技術・仕上がりの全平均,メニュー。料金の全平均,名前,性別、年齢、属性,投稿日時,総合,雰囲気,接客サービス,技術・仕上がり,メニュー・料金,口コミ本文,選択されたクーポン,メニューの種類,コメントへの返信,初回予約金額最小,初回予約金額最大
0,徳島,北島・藍住・松茂・鳴門,HIBIKA,https://beauty.hotpepper.jp/slnH000243953/?cstt=1,徳島県板野郡北島町中村字長池32-2,北島郵便局前、北島小学校隣,4400,6,1668,1459,6.0,【ご新規様限定！HIBIKA10周年記念】フルカラー＋カット＋トリートメント,10340,１０周年記念千田クーポン【春夏カラーとリラックス髪質改善トリートメント】,13000,10周年記念西山クーポン【カラーとスパの贅沢リラクゼーションコース】,17160,"¥10,000～¥10,999",43.0,40,532.0,1668.0,4.84,4.8,4.9,4.9,4.5,あさみさん,（女性/30代前半）,2023/5/20,5.0,4.0,5.0,5.0,4.0,今回もトリートメントでツヤツヤになり\n大満足です！ありがとうございました。♪\n綺麗にして...,千田クーポン【サロンでしか味わえない手触りに・・・。ハイダメージもOK】,[施術メニュー] トリートメント,無,10000,10999
1,徳島,北島・藍住・松茂・鳴門,HIBIKA,https://beauty.hotpepper.jp/slnH000243953/?cstt=1,徳島県板野郡北島町中村字長池32-2,北島郵便局前、北島小学校隣,4400,6,1668,1459,6.0,【ご新規様限定！HIBIKA10周年記念】フルカラー＋カット＋トリートメント,10340,１０周年記念千田クーポン【春夏カラーとリラックス髪質改善トリートメント】,13000,10周年記念西山クーポン【カラーとスパの贅沢リラクゼーションコース】,17160,"¥10,000～¥10,999",43.0,40,532.0,1668.0,4.84,4.8,4.9,4.9,4.5,すずくまさん,（女性/40代/パート・アルバイト）,2023/5/20,5.0,5.0,5.0,5.0,5.0,いつもこちらの希望をよく聞いていただけ、仕上がりの感じもとても良いので安心してお任せできます...,クーポン利用なし,[施術メニュー] カット,無,10000,10999
2,徳島,北島・藍住・松茂・鳴門,HIBIKA,https://beauty.hotpepper.jp/slnH000243953/?cstt=1,徳島県板野郡北島町中村字長池32-2,北島郵便局前、北島小学校隣,4400,6,1668,1459,6.0,【ご新規様限定！HIBIKA10周年記念】フルカラー＋カット＋トリートメント,10340,１０周年記念千田クーポン【春夏カラーとリラックス髪質改善トリートメント】,13000,10周年記念西山クーポン【カラーとスパの贅沢リラクゼーションコース】,17160,"¥10,000～¥10,999",43.0,40,532.0,1668.0,4.84,4.8,4.9,4.9,4.5,gonさん,（女性/20代前半/学生）,2023/5/19,5.0,5.0,5.0,5.0,5.0,カラーをお願いしました。\n髪色の相談を親身になって聞いてくださり、可愛く仕上げてもらうこと...,クーポン利用なし,[施術メニュー] カラー,無,10000,10999
3,徳島,北島・藍住・松茂・鳴門,HIBIKA,https://beauty.hotpepper.jp/slnH000243953/?cstt=1,徳島県板野郡北島町中村字長池32-2,北島郵便局前、北島小学校隣,4400,6,1668,1459,6.0,【ご新規様限定！HIBIKA10周年記念】フルカラー＋カット＋トリートメント,10340,１０周年記念千田クーポン【春夏カラーとリラックス髪質改善トリートメント】,13000,10周年記念西山クーポン【カラーとスパの贅沢リラクゼーションコース】,17160,"¥10,000～¥10,999",43.0,40,532.0,1668.0,4.84,4.8,4.9,4.9,4.5,はるさん,（女性/40代）,2023/5/19,4.0,4.0,5.0,5.0,3.0,今回も帰ってからの痒み等の不調はありません。\n軽くなったので、シャンプーや乾かすのが、楽に...,10周年記念坂田クーポン【むくみコリ疲れ、排出☆オールハンドすっきりスパ】,[施術メニュー] カット、ヘッドスパ、その他,有,10000,10999
4,徳島,北島・藍住・松茂・鳴門,HIBIKA,https://beauty.hotpepper.jp/slnH000243953/?cstt=1,徳島県板野郡北島町中村字長池32-2,北島郵便局前、北島小学校隣,4400,6,1668,1459,6.0,【ご新規様限定！HIBIKA10周年記念】フルカラー＋カット＋トリートメント,10340,１０周年記念千田クーポン【春夏カラーとリラックス髪質改善トリートメント】,13000,10周年記念西山クーポン【カラーとスパの贅沢リラクゼーションコース】,17160,"¥10,000～¥10,999",43.0,40,532.0,1668.0,4.84,4.8,4.9,4.9,4.5,やまさん,（女性/20代後半/会社員）,2023/5/18,4.0,4.0,4.0,4.0,3.0,以前担当スタイリストさんに施術いただいた仕上がりがとても良く、2回目もお願いしました。カウン...,【髪の広がりとカラーのダメージが気になる方へ】,[施術メニュー] カット、カラー、トリートメント,無,10000,10999
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
223233,福山・尾道,福山駅前・三吉周辺,Chou Chou Hair 【シュシュヘアー】,https://beauty.hotpepper.jp/slnH000400505/?cst...,広島県福山市木之庄町３-3-20,福山駅から徒歩15分【ファミリーマート木之庄店から徒歩1分】【髪質改善/縮毛矯正】,5400,2,0,12,1.0,,0,,0,,0,データ収集中,NaN,10,NaN,NaN,4.75,4.8,5.0,5.0,4.5,じゃむさん,（女性/10代後半）,2022/1/5,5.0,5.0,5.0,5.0,5.0,今日はありがとうございました！\n髪の毛のケアについて色々と丁寧に教えてもらい、家でもケアを...,クーポン利用なし,[施術メニュー] トリートメント、その他,有,23000,23999
223234,福山・尾道,福山駅前・三吉周辺,Chou Chou Hair 【シュシュヘアー】,https://beauty.hotpepper.jp/slnH000400505/?cst...,広島県福山市木之庄町３-3-20,福山駅から徒歩15分【ファミリーマート木之庄店から徒歩1分】【髪質改善/縮毛矯正】,5400,2,0,12,1.0,,0,,0,,0,データ収集中,NaN,10,NaN,NaN,4.75,4.8,5.0,5.0,4.5,しぃちゃんさん,（女性/20代前半/学生）,2021/11/15,5.0,5.0,5.0,5.0,4.0,初めての髪質改善だったけど、とても綺麗な仕上がりに驚きました。自分の髪とは信じれないほどサラ...,クーポン利用なし,[施術メニュー] カット、カラー、トリートメント,有,23000,23999
223235,福山・尾道,福山駅前・三吉周辺,Chou Chou Hair 【シュシュヘアー】,https://beauty.hotpepper.jp/slnH000400505/?cst...,広島県福山市木之庄町３-3-20,福山駅から徒歩15分【ファミリーマート木之庄店から徒歩1分】【髪質改善/縮毛矯正】,5400,2,0,12,1.0,,0,,0,,0,データ収集中,NaN,10,NaN,NaN,4.75,4.8,5.0,5.0,4.5,エマさん,（女性/20代前半/会社員）,2021/11/5,5.0,5.0,5.0,5.0,3.0,髪がとても綺麗になって感動しました!!\nお値段は高いけどそれ以上に効果がありました！\nぜ...,クーポン利用なし,[施術メニュー] カラー、トリートメント,有,23000,23999
223236,福山・尾道,福山駅前・三吉周辺,Chou Chou Hair 【シュシュヘアー】,https://b

In [25]:
df['2回目以降予約金額'].fillna(0, inplace=True)
df['2回目以降予約金額'].isna().sum()

0

In [26]:
_list_for_df = []
for _list in df['2回目以降予約金額'].str.replace('¥', '').str.replace(',', '').str.split('～'):
    if  _list is np.nan or len(_list) < 2:
        _list=['0', '0']
    _list_for_df.append(_list)

In [27]:
df = df.reset_index(drop=True)

In [28]:
df = pd.concat([df, pd.DataFrame(_list_for_df, columns=['2回目以降予約金額最小', '2回目以降予約金額最大'])], axis=1)

In [29]:
df.drop(['2回目以降予約金額'], axis=1, inplace=True)

In [30]:
df.tail(1)

,県,エリア,サロン名,URL,住所,アクセス,カット料金,セット面の数,ブログ投稿数,口コミ数,スタッフ数,1番人気クーポン名,1番人気クーポン価格,2番人気クーポン名,2番人気クーポン価格,3番人気クーポン名,3番人気クーポン価格,クーポン数,メニュー数,スタイル数,ブログ数,口コミスコアサロン全平均,雰囲気の全平均,接客サービスの全平均,技術・仕上がりの全平均,メニュー。料金の全平均,名前,性別、年齢、属性,投稿日時,総合,雰囲気,接客サービス,技術・仕上がり,メニュー・料金,口コミ本文,選択されたクーポン,メニューの種類,コメントへの返信,初回予約金額最小,初回予約金額最大,2回目以降予約金額最小,2回目以降予約金額最大
223237,福山・尾道,福山駅前・三吉周辺,Chou Chou Hair 【シュシュヘアー】,https://beauty.hotpepper.jp/slnH000400505/?cst...,広島県福山市木之庄町３-3-20,福山駅から徒歩15分【ファミリーマート木之庄店から徒歩1分】【髪質改善/縮毛矯正】,5400,2,0,12,1.0,,0,,0,,0,NaN,10,NaN,NaN,4.75,4.8,5.0,5.0,4.5,きいさん,（女性/20代前半/学生）,2021/7/3,5.0,5.0,5.0,5.0,4.0,初めての髪質改善を行って貰いました。\n最初にどこが傷んでいるかなど詳しくお話をしていただき...,クーポン利用なし,[施術メニュー] トリートメント、その他,有,23000,23999,0,0


# 処理したカラム
- エリア
- 投稿日時
- セット面の数
- カット料金
- ブログ投稿数
- 口コミ数
- 1〜3番人気クーポン名、価格
- 初回、2回目以降予約金額

In [31]:
df['クーポン数'] = df['クーポン数'].fillna(0).astype(int)

In [32]:
df['口コミ数'] = df['口コミ数'].astype(int)

In [33]:
df['カット料金'] = df['カット料金'].str.replace('-', '0')
df['カット料金'] = df['カット料金'].astype(int)

In [34]:
df['セット面の数'] = df['セット面の数'].str.replace('-', '0')
df['セット面の数'] = df['セット面の数'].astype(int)

In [35]:
df['1番人気クーポン価格'] = df['1番人気クーポン価格'].astype(str).str.replace('‐', '0')
df['2番人気クーポン価格'] = df['2番人気クーポン価格'].astype(str).str.replace('‐', '0')
df['3番人気クーポン価格'] = df['3番人気クーポン価格'].astype(str).str.replace('‐', '0')

In [36]:
df['ブログ投稿数'] = df['ブログ投稿数'].astype(int)

In [37]:
df.columns

Index(['県', 'エリア', 'サロン名', 'URL', '住所', 'アクセス', 'カット料金', 'セット面の数', 'ブログ投稿数',
       '口コミ数', 'スタッフ数', '1番人気クーポン名', '1番人気クーポン価格', '2番人気クーポン名', '2番人気クーポン価格',
       '3番人気クーポン名', '3番人気クーポン価格', 'クーポン数', 'メニュー数', 'スタイル数', 'ブログ数',
       '口コミスコアサロン全平均', '雰囲気の全平均', '接客サービスの全平均', '技術・仕上がりの全平均', 'メニュー。料金の全平均',
       '名前', '性別、年齢、属性', '投稿日時', '総合', '雰囲気', '接客サービス', '技術・仕上がり', 'メニュー・料金',
       '口コミ本文', '選択されたクーポン', 'メニューの種類', 'コメントへの返信', '初回予約金額最小', '初回予約金額最大',
       '2回目以降予約金額最小', '2回目以降予約金額最大'],
      dtype='object')

In [38]:
df[['スタイル数', 'ブログ数',
       '口コミスコアサロン全平均', '雰囲気の全平均', '接客サービスの全平均', '技術・仕上がりの全平均', 'メニュー。料金の全平均',
       '総合', '雰囲気', '接客サービス', '技術・仕上がり', 'メニュー・料金',]]=\
df[['スタイル数', 'ブログ数',
       '口コミスコアサロン全平均', '雰囲気の全平均', '接客サービスの全平均', '技術・仕上がりの全平均', 'メニュー。料金の全平均',
       '総合', '雰囲気', '接客サービス', '技術・仕上がり', 'メニュー・料金',]].fillna(0)

In [39]:
df[['スタイル数', 'ブログ数', '総合', '雰囲気', '接客サービス', '技術・仕上がり', 'メニュー・料金',]]=\
df[['スタイル数', 'ブログ数', '総合', '雰囲気', '接客サービス', '技術・仕上がり', 'メニュー・料金',]].astype(int)

In [40]:
df[['スタイル数', 'ブログ数',
       '口コミスコアサロン全平均', '雰囲気の全平均', '接客サービスの全平均', '技術・仕上がりの全平均', 'メニュー。料金の全平均',
       '総合', '雰囲気', '接客サービス', '技術・仕上がり', 'メニュー・料金',]].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 223238 entries, 0 to 223237
Data columns (total 12 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   スタイル数         223238 non-null  int64 
 1   ブログ数          223238 non-null  int64 
 2   口コミスコアサロン全平均  223238 non-null  object
 3   雰囲気の全平均       223238 non-null  object
 4   接客サービスの全平均    223238 non-null  object
 5   技術・仕上がりの全平均   223238 non-null  object
 6   メニュー。料金の全平均   223238 non-null  object
 7   総合            223238 non-null  int64 
 8   雰囲気           223238 non-null  int64 
 9   接客サービス        223238 non-null  int64 
 10  技術・仕上がり       223238 non-null  int64 
 11  メニュー・料金       223238 non-null  int64 
dtypes: int64(7), object(5)
memory usage: 20.4+ MB


In [41]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 223238 entries, 0 to 223237
Data columns (total 42 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   県             223238 non-null  object 
 1   エリア           223238 non-null  object 
 2   サロン名          223238 non-null  object 
 3   URL           223238 non-null  object 
 4   住所            223238 non-null  object 
 5   アクセス          223238 non-null  object 
 6   カット料金         223238 non-null  int64  
 7   セット面の数        223238 non-null  int64  
 8   ブログ投稿数        223238 non-null  int64  
 9   口コミ数          223238 non-null  int64  
 10  スタッフ数         223189 non-null  float64
 11  1番人気クーポン名     223238 non-null  object 
 12  1番人気クーポン価格    223238 non-null  object 
 13  2番人気クーポン名     223238 non-null  object 
 14  2番人気クーポン価格    223238 non-null  object 
 15  3番人気クーポン名     223238 non-null  object 
 16  3番人気クーポン価格    223238 non-null  object 
 17  クーポン数         223238 non-null  int64  
 18  メニュー

In [42]:
!pwd

/Users/masudaniwabinari/Desktop/hpb_scraping


In [43]:
df.to_csv('一時保存.csv', index=False)

In [44]:
df['性別、年齢、属性'].unique()

array(['（女性/30代前半）', '（女性/40代/パート・アルバイト）', '（女性/20代前半/学生）', '（女性/40代）',
       '（女性/20代後半/会社員）', '（男性/40代/会社員）', '（女性/20代前半）', '（女性/40代/会社員）',
       '（女性/30代前半/その他）', '（男性/30代前半/その他）', '（男性/30代前半/会社員）',
       '（女性/10代後半/学生）', '（女性/30代後半/主婦）', '（女性/30代前半/会社員）', '（男性/60代/自営業）',
       '（女性/30代後半/会社員）', '（女性/20代前半/会社員）', '（女性/20代後半）', '（女性/30代前半/自営業）',
       '（男性/50代）', '（女性/50代）', '（男性/20代前半/学生）', '（女性/50代/その他）',
       '（女性/50代/会社員）', '（女性/30代後半/その他）', '（女性/60代/主婦）', '（女性/30代後半）',
       '（女性/10代後半）', '（女性/30代前半/主婦）', '（女性/20代後半/学生）', '（女性/50代/自営業）',
       '（男性/50代/会社員）', '（女性/50代/主婦）', '（女性/40代/主婦）', '（女性/50代/パート・アルバイト）',
       '（女性/40代/その他）', '（男性/40代）', '（男性/10代後半/パート・アルバイト）',
       '（女性/20代後半/パート・アルバイト）', '（女性/30代後半/パート・アルバイト）', '（男性/20代前半）',
       '（女性/20代後半/主婦）', '（男性/30代前半）', '（男性/10代後半/会社員）', '（男性/20代前半/会社員）',
       '（女性/60代）', '（女性/40代/自営業）', '（女性/30代前半/パート・アルバイト）',
       '（女性/30代後半/自営業）', '（女性/20代前半/パート・アルバイト）', '（男性/50代/自営業）',
       '（女性/40代/学生）', '（女性/20代前半/その他）', 

In [45]:
_list_for_df = []
for _list in df['性別、年齢、属性'].str.replace('（', '').str.replace('）', '').str.split('/'):
    if _list is np.nan:
        _list = ['不明', '不明', '不明']
    elif len(_list) == 1:
        _list.extend(['不明', '不明'])
    elif len(_list) == 2:
        _list.append('不明')
    _list_for_df.append(_list)

In [46]:
df = df.reset_index(drop=True)

In [47]:
df = pd.concat([df,pd.DataFrame(_list_for_df, columns=['性別', '年齢', '職業'])], axis=1)

In [48]:
df.drop('性別、年齢、属性', axis=1, inplace=True)

In [49]:
df.tail()

,県,エリア,サロン名,URL,住所,アクセス,カット料金,セット面の数,ブログ投稿数,口コミ数,スタッフ数,1番人気クーポン名,1番人気クーポン価格,2番人気クーポン名,2番人気クーポン価格,3番人気クーポン名,3番人気クーポン価格,クーポン数,メニュー数,スタイル数,ブログ数,口コミスコアサロン全平均,雰囲気の全平均,接客サービスの全平均,技術・仕上がりの全平均,メニュー。料金の全平均,名前,投稿日時,総合,雰囲気,接客サービス,技術・仕上がり,メニュー・料金,口コミ本文,選択されたクーポン,メニューの種類,コメントへの返信,初回予約金額最小,初回予約金額最大,2回目以降予約金額最小,2回目以降予約金額最大,性別,年齢,職業
223233,福山・尾道,福山駅前・三吉周辺,Chou Chou Hair 【シュシュヘアー】,https://beauty.hotpepper.jp/slnH000400505/?cst...,広島県福山市木之庄町３-3-20,福山駅から徒歩15分【ファミリーマート木之庄店から徒歩1分】【髪質改善/縮毛矯正】,5400,2,0,12,1.0,,0,,0,,0,0,10,0,0,4.75,4.8,5.0,5.0,4.5,じゃむさん,2022/1/5,5,5,5,5,5,今日はありがとうございました！\n髪の毛のケアについて色々と丁寧に教えてもらい、家でもケアを...,クーポン利用なし,[施術メニュー] トリートメント、その他,有,23000,23999,0,0,女性,10代後半,不明
223234,福山・尾道,福山駅前・三吉周辺,Chou Chou Hair 【シュシュヘアー】,https://beauty.hotpepper.jp/slnH000400505/?cst...,広島県福山市木之庄町３-3-20,福山駅から徒歩15分【ファミリーマート木之庄店から徒歩1分】【髪質改善/縮毛矯正】,5400,2,0,12,1.0,,0,,0,,0,0,10,0,0,4.75,4.8,5.0,5.0,4.5,しぃちゃんさん,2021/11/15,5,5,5,5,4,初めての髪質改善だったけど、とても綺麗な仕上がりに驚きました。自分の髪とは信じれないほどサラ...,クーポン利用なし,[施術メニュー] カット、カラー、トリートメント,有,23000,23999,0,0,女性,20代前半,学生
223235,福山・尾道,福山駅前・三吉周辺,Chou Chou Hair 【シュシュヘアー】,https://beauty.hotpepper.jp/slnH000400505/?cst...,広島県福山市木之庄町３-3-20,福山駅から徒歩15分【ファミリーマート木之庄店から徒歩1分】【髪質改善/縮毛矯正】,5400,2,0,12,1.0,,0,,0,,0,0,10,0,0,4.75,4.8,5.0,5.0,4.5,エマさん,2021/11/5,5,5,5,5,3,髪がとても綺麗になって感動しました!!\nお値段は高いけどそれ以上に効果がありました！\nぜ...,クーポン利用なし,[施術メニュー] カラー、トリートメント,有,23000,23999,0,0,女性,20代前半,会社員
223236,福山・尾道,福山駅前・三吉周辺,Chou Chou Hair 【シュシュヘアー】,https://beauty.hotpepper.jp/slnH000400505/?cst...,広島県福山市木之庄町３-3-20,福山駅から徒歩15分【ファミリーマート木之庄店から徒歩1分】【髪質改善/縮毛矯正】,5400,2,0,12,1.0,,0,,0,,0,0,10,0,0,4.75,4.8,5.0,5.0,4.5,サラさん,2021/8/19,5,5,5,5,5,初めての髪質改善だったけど最初の触り心地と終わってからの触り心地が全く違って驚きました。\n...,クーポン利用なし,[施術メニュー] トリートメント、その他,有,23000,23999,0,0,女性,10代後半,学生
223237,福山・尾道,福山駅前・三吉周辺,Chou Chou Hair 【シュシュヘアー】,https://beauty.hotpepper.jp/slnH000400505/?cst...,広島県福山市木之庄町３-3-20,福山駅から徒歩15分【ファミリーマート木之庄店から徒歩1分】【髪質改善/縮毛矯正】,5400,2,0,12,1.0,,0,,0,,0,0,10,0,0,4.75,4.8,5.0,5.0,4.5,きいさん,2021/7/3,5,5,5,5,4,初めての髪質改善を行って貰いました。\n最初にどこが傷んでいるかなど詳しくお話をしていただき...,クーポン利用なし,[施術メニュー] トリートメント、その他,有,23000,23999,0,0,女性,20代前半,学生


# 処理したカラム
- エリア
- 投稿日時
- セット面の数
- カット料金
- ブログ投稿数
- 口コミ数
- 1〜3番人気クーポン名、価格
- 初回、2回目以降予約金額
- 性別、年齢、属性

In [50]:
df['駅徒歩'] = df['アクセス'].apply(lambda x: mojimoji.zen_to_han(x))

In [63]:
import re
code_regex = re.compile('[!"#$%&\'\\\\()*+,-./:;<=>?@[\\]^_`{|}~「」〔〕“”〈〉『』【】＆＊・（）＄＃＠。、？！｀＋￥％♪]')

_list=[]
i=1
for row in df['駅徒歩']:
    if len(row) < 3:
        print(row)
    if '徒歩' in row and '分' in row and '駅':
        row = row.replace('　', '').replace(' ', '')
        row = code_regex.sub('', row)
        
        row_list = re.split('\D+', row)
        for row in row_list:
            if row != '':
                if len(row) <= 2:
                    row = row
                    break
            
        if 'O' in row or 'o' in row:
            row = 0
            
        if row != 0:
            row = re.sub(r'\D', '', row)
        
        if row == '':
            row = pd.NA

    else:
        row = pd.NA
        # print(row)
    _list.append(row)
    i+=1
    
_Series = pd.Series(_list)
df['駅徒歩'] = _Series

In [59]:
df['駅徒歩'].unique()

array([<NA>, '3', '4', '5', 0, '10', '8', '1', '15', '7', '2', '30', '9',
       '6', '18', '17', '12', '20', '1520', '23', '0', '45', '25', '11',
       '14', '19', '28', '38', '22', '57', '60', '50', '40', '2126',
       '301', '082231123023', '16', '52', '13', '110', '3504', '315',
       '145', '21', '513', '710', '51', '6007', '113', '112', '62', '31',
       '1315'], dtype=object)

In [60]:
!pip freeze | grep pandas

pandas==1.4.1


In [61]:
df['駅徒歩'] = np.floor(pd.to_numeric(df['駅徒歩'], errors='coerce')).astype('Int64')

In [62]:
df['駅徒歩']

0         <NA>
1         <NA>
2         <NA>
3         <NA>
4         <NA>
          ... 
223233      15
223234      15
223235      15
223236      15
223237      15
Name: 駅徒歩, Length: 223238, dtype: Int64

In [52]:
df['駅徒歩'].isna().sum()

1158

# 処理したカラム
- エリア
- 投稿日時
- セット面の数
- カット料金
- ブログ投稿数
- 口コミ数
- 1〜3番人気クーポン名、価格
- 初回、2回目以降予約金額
- 性別、年齢、属性
- アクセス

In [53]:
df.to_csv('一時保存.csv', index=False)

In [54]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17796 entries, 0 to 17795
Data columns (total 45 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   県             17796 non-null  object 
 1   エリア           17796 non-null  object 
 2   サロン名          17796 non-null  object 
 3   URL           17796 non-null  object 
 4   住所            17796 non-null  object 
 5   アクセス          17796 non-null  object 
 6   カット料金         17796 non-null  int64  
 7   セット面の数        17796 non-null  int64  
 8   ブログ投稿数        17796 non-null  int64  
 9   口コミ数          17796 non-null  int64  
 10  スタッフ数         17793 non-null  float64
 11  1番人気クーポン名     17796 non-null  object 
 12  1番人気クーポン価格    17796 non-null  object 
 13  2番人気クーポン名     17796 non-null  object 
 14  2番人気クーポン価格    17796 non-null  object 
 15  3番人気クーポン名     17796 non-null  object 
 16  3番人気クーポン価格    17796 non-null  object 
 17  クーポン数         17796 non-null  int64  
 18  メニュー数         17796 non-nu

In [55]:
df['メニューの種類']

0                         [施術メニュー] カット、カラー
1                         [施術メニュー] カット、カラー
2                         [施術メニュー] カット、カラー
3                 [施術メニュー] カット、カラー、トリートメント
4        [施術メニュー] カット、カラー、パーマ、縮毛矯正、トリートメント
                       ...                
17791            [施術メニュー] 縮毛矯正、トリートメント、その他
17792            [施術メニュー] 縮毛矯正、トリートメント、その他
17793                 [施術メニュー] カラー、トリートメント
17794                     [施術メニュー] カット、カラー
17795                         [施術メニュー] カット
Name: メニューの種類, Length: 17796, dtype: object

In [56]:
df['メニューの種類'].str.replace(' ', '').str.replace(' ', '').str.replace('[施術メニュー]', '', regex=False)
# .str.replace('[施術メニュー]', '').str.split('、')

0                         カット、カラー
1                         カット、カラー
2                         カット、カラー
3                 カット、カラー、トリートメント
4        カット、カラー、パーマ、縮毛矯正、トリートメント
                   ...           
17791            縮毛矯正、トリートメント、その他
17792            縮毛矯正、トリートメント、その他
17793                 カラー、トリートメント
17794                     カット、カラー
17795                         カット
Name: メニューの種類, Length: 17796, dtype: object

In [57]:
df['メニューの種類'] = df['メニューの種類'].str.replace(' ', '').str.replace(' ', '').str.replace('[施術メニュー]', '', regex=False)

In [58]:
df['メニューの種類'] = df['メニューの種類'].fillna('メニュー無し')

In [59]:
_menu_list = []
for row in df['メニューの種類'].unique():
    for menu_row in row.split('、'):
        if not menu_row in _menu_list:
            _menu_list.append(menu_row)
_menu_list

['カット',
 'カラー',
 'トリートメント',
 'パーマ',
 '縮毛矯正',
 'その他',
 'ヘッドスパ',
 'ヘアセット',
 'エクステ',
 '着付け',
 'メニュー無し']

In [60]:
len(_menu_list)

11

In [61]:
df['メニューの種類']

0                         カット、カラー
1                         カット、カラー
2                         カット、カラー
3                 カット、カラー、トリートメント
4        カット、カラー、パーマ、縮毛矯正、トリートメント
                   ...           
17791            縮毛矯正、トリートメント、その他
17792            縮毛矯正、トリートメント、その他
17793                 カラー、トリートメント
17794                     カット、カラー
17795                         カット
Name: メニューの種類, Length: 17796, dtype: object

In [62]:
len(df)

17796

In [63]:
_cut_list=[]
_color_list=[]
_treatment_list=[]
_pama_list=[]
_straightening_list=[]
_other_list=[]
_head_spa_list=[]
_hair_set_list=[]
_extension_list=[]
_dressing_list=[]
_nothing_list=[]
for row in df['メニューの種類']:
    
    if 'カット' in row.split('、'):
        cut = 1
    else:
        cut = 0
    _cut_list.append(cut)
    
    if 'カラー' in row.split('、'):
        color=1
    else:
        color=0
    _color_list.append(color)
    
    if 'トリートメント' in row.split('、'):
        treatment=1
    else:
        treatment=0
    _treatment_list.append(treatment)
    
    if 'パーマ' in row.split('、'):
        pama=1
    else:
        pama=0
    _pama_list.append(pama)
    
    if '縮毛矯正' in row.split('、'):
        straightening=1
    else:
        straightening=0
    _straightening_list.append(straightening)
    
    if 'その他' in row.split('、'):
        other=1
    else:
        other=0
    _other_list.append(other)
    
    if 'ヘッドスパ' in row.split('、'):
        head_spa=1
    else:
        head_spa=0
    _head_spa_list.append(head_spa)
        
    if 'ヘアセット' in row.split('、'):
        hair_set=1
    else:
        hair_set=0
    _hair_set_list.append(hair_set)
        
    if 'エクステ' in row.split('、'):
        extension=1
    else:
        extension=0
    _extension_list.append(extension)

    if '着付け' in row.split('、'):
        dressing=1
    else:
        dressing=0
    _dressing_list.append(dressing)
        
    if 'メニュー無し' in row.split('、'):
        nothing=1
    else:
        nothing=0
    _nothing_list.append(nothing)
    
_df = pd.DataFrame({'カット選択': _cut_list,
             'カラー選択': _color_list,
             'トリートメント選択': _treatment_list,
             'パーマ選択': _pama_list,
             '縮毛矯正選択': _straightening_list,
             'その他選択': _other_list,
             'ヘッドスパ選択': _head_spa_list,
             'ヘアセット選択': _hair_set_list,
             'エクステ選択': _extension_list,
             '着付け選択': _dressing_list,
             'メニュー無し選択': _nothing_list})

In [64]:
_df

,カット選択,カラー選択,トリートメント選択,パーマ選択,縮毛矯正選択,その他選択,ヘッドスパ選択,ヘアセット選択,エクステ選択,着付け選択,メニュー無し選択
0,1,1,0,0,0,0,0,0,0,0,0
1,1,1,0,0,0,0,0,0,0,0,0
2,1,1,0,0,0,0,0,0,0,0,0
3,1,1,1,0,0,0,0,0,0,0,0
4,1,1,1,1,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...
17791,0,0,1,0,1,1,0,0,0,0,0
17792,0,0,1,0,1,1,0,0,0,0,0
17793,0,1,1,0,0,0,0,0,0,0,0
17794,1,1,0,0,0,0,0,0,0,0,0


In [65]:
df = df.reset_index(drop=True)

In [66]:
df = pd.concat(objs=[df, _df,], axis=1)
df

,県,エリア,サロン名,URL,住所,アクセス,カット料金,セット面の数,ブログ投稿数,口コミ数,スタッフ数,1番人気クーポン名,1番人気クーポン価格,2番人気クーポン名,2番人気クーポン価格,3番人気クーポン名,3番人気クーポン価格,クーポン数,メニュー数,スタイル数,ブログ数,口コミスコアサロン全平均,雰囲気の全平均,接客サービスの全平均,技術・仕上がりの全平均,メニュー。料金の全平均,名前,投稿日時,総合,雰囲気,接客サービス,技術・仕上がり,メニュー・料金,口コミ本文,選択されたクーポン,メニューの種類,コメントへの返信,初回予約金額最小,初回予約金額最大,2回目以降予約金額最小,2回目以降予約金額最大,性別,年齢,職業,駅徒歩,カット選択,カラー選択,トリートメント選択,パーマ選択,縮毛矯正選択,その他選択,ヘッドスパ選択,ヘアセット選択,エクステ選択,着付け選択,メニュー無し選択
0,福岡,薬院・平尾・高宮,Lu’ce coco【ルーチェ ココ】,https://beauty.hotpepper.jp/slnH000284489/?cstt=1,福岡県福岡市南区大楠３-12-21モナーク高宮1F,西鉄高宮駅徒歩4分 インナーカラー/髪質改善/キッズカット/イルミナカラー/縮毛矯正,3850,8,1346,233,8.0,【透明感UP☆】カット＋イルミナカラー＋オージュアリペアTR¥14300→9350,9350,【透明感UP☆】カット＋イルミナカラー＋オージュアリペアTR￥14300→￥8800,8800,【透明感UP☆】カット＋イルミナカラー＋オージュアリペアTR¥14300→￥11000,11000,51,35,138,1346,4.60,4.6,4.7,4.6,4.4,siinaさん,2023/4/22,5,5,5,5,5,2回目の利用で、今回はイルミナカラーとカットをお願いしました！数年ぶりに前髪を作りたいと思い...,【透明感UP☆】カット＋イルミナカラー＋オージュアリペアTR\14300→\9900,カット、カラー,無,8000,8999,8000,8999,女性,30代前半,パート・アルバイト,4,1,1,0,0,0,0,0,0,0,0,0
1,福岡,薬院・平尾・高宮,Lu’ce coco【ルーチェ ココ】,https://beauty.hotpepper.jp/slnH000284489/?cstt=1,福岡県福岡市南区大楠３-12-21モナーク高宮1F,西鉄高宮駅徒歩4分 インナーカラー/髪質改善/キッズカット/イルミナカラー/縮毛矯正,3850,8,1346,233,8.0,【透明感UP☆】カット＋イルミナカラー＋オージュアリペアTR¥14300→9350,9350,【透明感UP☆】カット＋イルミナカラー＋オージュアリペアTR￥14300→￥8800,8800,【透明感UP☆】カット＋イルミナカラー＋オージュアリペアTR¥14300→￥11000,11000,51,35,138,1346,4.60,4.6,4.7,4.6,4.4,まなみんさん,2023/4/18,4,5,4,5,5,いろいろ説明してもらって説明の仕方もわかりやすかったので有り難かったです!!\n途中でカラー...,クーポン利用なし,カット、カラー,無,8000,8999,8000,8999,女性,30代後半,パート・アルバイト,4,1,1,0,0,0,0,0,0,0,0,0
2,福岡,薬院・平尾・高宮,Lu’ce coco【ルーチェ ココ】,https://beauty.hotpepper.jp/slnH000284489/?cstt=1,福岡県福岡市南区大楠３-12-21モナーク高宮1F,西鉄高宮駅徒歩4分 インナーカラー/髪質改善/キッズカット/イルミナカラー/縮毛矯正,3850,8,1346,233,8.0,【透明感UP☆】カット＋イルミナカラー＋オージュアリペアTR¥14300→9350,9350,【透明感UP☆】カット＋イルミナカラー＋オージュアリペアTR￥14300→￥8800,8800,【透明感UP☆】カット＋イルミナカラー＋オージュアリペアTR¥14300→￥11000,11000,51,35,138,1346,4.60,4.6,4.7,4.6,4.4,けんけんさん,2023/4/13,5,5,5,5,5,毎回いい感じにしていただいてありがとうございます,【大人気☆】メンズカット＋カラー＋シャンプー\12100→￥7700,カット、カラー,無,8000,8999,8000,8999,男性,20代後半,不明,4,1,1,0,0,0,0,0,0,0,0,0
3,福岡,薬院・平尾・高宮,Lu’ce coco【ルーチェ ココ】,https://beauty.hotpepper.jp/slnH000284489/?cstt=1,福岡県福岡市南区大楠３-12-21モナーク高宮1F,西鉄高宮駅徒歩4分 インナーカラー/髪質改善/キッズカット/イルミナカラー/縮毛矯正,3850,8,1346,233,8.0,【透明感UP☆】カット＋イルミナカラー＋オージュアリペアTR¥14300→9350,9350,【透明感UP☆】カット＋イルミナカラー＋オージュアリペアTR￥14300→￥8800,8800,【透明感UP☆】カット＋イルミナカラー＋オージュアリペアTR¥14300→￥11000,11000,51,35,138,1346,4.60,4.6,4.7,4.6,4.4,LGMさん,2023/4/4,2,3,3,2,2,キッズカットをお願いしたのですが後ろの真ん中長い野がってガタガタです…。立たせて切ってて娘も...,【ママさん限定】カット＋カラー＋TR & [キッズカット無料] \12100→￥8250,カット、カラー、トリートメント,無,8000,8999,8000,8999,女性,30代後半,主婦,4,1,1,1,0,0,0,0,0,0,0,0
4,福岡,薬院・平尾・高宮,Lu’ce coco【ルーチェ ココ】,https://beauty.hotpepper.jp/slnH000284489/?cstt=1,福岡県福岡市南区大楠３-12-21モナーク高宮1F,西鉄高宮駅徒歩4分 インナーカラー/髪質改善/キッズカット/イルミナカラー/縮毛矯正,3850,8,1346,233,8.0,【透明感UP☆】カット＋イルミナカラー＋オージュアリペアTR¥14300→9350,9350,【透明感UP☆】カット＋イルミナカラー＋オージュアリペアTR￥14300→￥8800,8800,【透明感UP☆】カット＋イルミナカラー＋オージュアリペアTR¥14300→￥11000,11000,51,35,138,1346,4.60,4.6,4.7,4.6,4.4,ゆうきさん,2023/3/19,4,4,5,4,4,先日は口うるさい私の注文にお付き合い頂き、本当にありがとうございました!\n長年通っていた美...,カット+イルミナカラー+デジタルパーマ+オージュアリペアTR\26950→ \17600,カット、カラー、パーマ、縮毛矯正、トリートメント,無,8000,8999,8000,8999,女性,40代,その他,4,1,1,1,1,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17791,福岡,薬院・平尾・高宮,la paix【ラペ】,https://beauty.hotpepper.jp/slnH000050281/?cst...,福岡県福岡市中央区薬院１－１６－１８ 江島ビル３０５号,西鉄・地下鉄薬院駅 徒歩3分,4860,3,0,3,3.0,,0,,0,,0,0,35,0,0,5.00,5.0,5.0,5.0,4.0,みさきさん,2022/11/18,5,5,5,5,4,先日縮毛矯正を初めてかけてもらいました。\nとってもサラサラで髪の毛が生まれ変わったようで毎...,クーポン利用なし,縮毛矯正、トリートメント、その他,無,17000,17999,12000,12999,女性,30代前半,会社員,3,0,0,1,0,1,1,0,0,0,0,0
17792,福岡,薬院・平尾・高宮,la paix【ラペ】,https://beauty.hotpepper.jp/slnH000050281/?cst...,福岡県福岡市中央区薬院１－１６－１８ 江島ビル３０５号,西鉄・地下鉄薬院駅 徒歩3分,4860,3,0,3,3.0,,0,,0,,0,0,35,0,0,5.00,5.0,5.0,5.0,4.0,みさきさん,2022/4/23,5,5,5,5,5,去年から気になっててやっと伺うことが出来ました。\nLINEでのやり取りからすごく丁寧で安心...,クーポン利用なし,縮毛矯正、トリートメント、その他,無,17000,17999,12000,12999,女性,20代前半,会社員,3,0,0,1,0,1,1,0,0,0,0,0
17793,福岡,薬院・

In [67]:
df['投稿日時'] = df['投稿日時'].str.replace('　', '').str.replace(' ', '').str.replace('[投稿日]', '', regex=False)

In [68]:
df

,県,エリア,サロン名,URL,住所,アクセス,カット料金,セット面の数,ブログ投稿数,口コミ数,スタッフ数,1番人気クーポン名,1番人気クーポン価格,2番人気クーポン名,2番人気クーポン価格,3番人気クーポン名,3番人気クーポン価格,クーポン数,メニュー数,スタイル数,ブログ数,口コミスコアサロン全平均,雰囲気の全平均,接客サービスの全平均,技術・仕上がりの全平均,メニュー。料金の全平均,名前,投稿日時,総合,雰囲気,接客サービス,技術・仕上がり,メニュー・料金,口コミ本文,選択されたクーポン,メニューの種類,コメントへの返信,初回予約金額最小,初回予約金額最大,2回目以降予約金額最小,2回目以降予約金額最大,性別,年齢,職業,駅徒歩,カット選択,カラー選択,トリートメント選択,パーマ選択,縮毛矯正選択,その他選択,ヘッドスパ選択,ヘアセット選択,エクステ選択,着付け選択,メニュー無し選択
0,福岡,薬院・平尾・高宮,Lu’ce coco【ルーチェ ココ】,https://beauty.hotpepper.jp/slnH000284489/?cstt=1,福岡県福岡市南区大楠３-12-21モナーク高宮1F,西鉄高宮駅徒歩4分 インナーカラー/髪質改善/キッズカット/イルミナカラー/縮毛矯正,3850,8,1346,233,8.0,【透明感UP☆】カット＋イルミナカラー＋オージュアリペアTR¥14300→9350,9350,【透明感UP☆】カット＋イルミナカラー＋オージュアリペアTR￥14300→￥8800,8800,【透明感UP☆】カット＋イルミナカラー＋オージュアリペアTR¥14300→￥11000,11000,51,35,138,1346,4.60,4.6,4.7,4.6,4.4,siinaさん,2023/4/22,5,5,5,5,5,2回目の利用で、今回はイルミナカラーとカットをお願いしました！数年ぶりに前髪を作りたいと思い...,【透明感UP☆】カット＋イルミナカラー＋オージュアリペアTR\14300→\9900,カット、カラー,無,8000,8999,8000,8999,女性,30代前半,パート・アルバイト,4,1,1,0,0,0,0,0,0,0,0,0
1,福岡,薬院・平尾・高宮,Lu’ce coco【ルーチェ ココ】,https://beauty.hotpepper.jp/slnH000284489/?cstt=1,福岡県福岡市南区大楠３-12-21モナーク高宮1F,西鉄高宮駅徒歩4分 インナーカラー/髪質改善/キッズカット/イルミナカラー/縮毛矯正,3850,8,1346,233,8.0,【透明感UP☆】カット＋イルミナカラー＋オージュアリペアTR¥14300→9350,9350,【透明感UP☆】カット＋イルミナカラー＋オージュアリペアTR￥14300→￥8800,8800,【透明感UP☆】カット＋イルミナカラー＋オージュアリペアTR¥14300→￥11000,11000,51,35,138,1346,4.60,4.6,4.7,4.6,4.4,まなみんさん,2023/4/18,4,5,4,5,5,いろいろ説明してもらって説明の仕方もわかりやすかったので有り難かったです!!\n途中でカラー...,クーポン利用なし,カット、カラー,無,8000,8999,8000,8999,女性,30代後半,パート・アルバイト,4,1,1,0,0,0,0,0,0,0,0,0
2,福岡,薬院・平尾・高宮,Lu’ce coco【ルーチェ ココ】,https://beauty.hotpepper.jp/slnH000284489/?cstt=1,福岡県福岡市南区大楠３-12-21モナーク高宮1F,西鉄高宮駅徒歩4分 インナーカラー/髪質改善/キッズカット/イルミナカラー/縮毛矯正,3850,8,1346,233,8.0,【透明感UP☆】カット＋イルミナカラー＋オージュアリペアTR¥14300→9350,9350,【透明感UP☆】カット＋イルミナカラー＋オージュアリペアTR￥14300→￥8800,8800,【透明感UP☆】カット＋イルミナカラー＋オージュアリペアTR¥14300→￥11000,11000,51,35,138,1346,4.60,4.6,4.7,4.6,4.4,けんけんさん,2023/4/13,5,5,5,5,5,毎回いい感じにしていただいてありがとうございます,【大人気☆】メンズカット＋カラー＋シャンプー\12100→￥7700,カット、カラー,無,8000,8999,8000,8999,男性,20代後半,不明,4,1,1,0,0,0,0,0,0,0,0,0
3,福岡,薬院・平尾・高宮,Lu’ce coco【ルーチェ ココ】,https://beauty.hotpepper.jp/slnH000284489/?cstt=1,福岡県福岡市南区大楠３-12-21モナーク高宮1F,西鉄高宮駅徒歩4分 インナーカラー/髪質改善/キッズカット/イルミナカラー/縮毛矯正,3850,8,1346,233,8.0,【透明感UP☆】カット＋イルミナカラー＋オージュアリペアTR¥14300→9350,9350,【透明感UP☆】カット＋イルミナカラー＋オージュアリペアTR￥14300→￥8800,8800,【透明感UP☆】カット＋イルミナカラー＋オージュアリペアTR¥14300→￥11000,11000,51,35,138,1346,4.60,4.6,4.7,4.6,4.4,LGMさん,2023/4/4,2,3,3,2,2,キッズカットをお願いしたのですが後ろの真ん中長い野がってガタガタです…。立たせて切ってて娘も...,【ママさん限定】カット＋カラー＋TR & [キッズカット無料] \12100→￥8250,カット、カラー、トリートメント,無,8000,8999,8000,8999,女性,30代後半,主婦,4,1,1,1,0,0,0,0,0,0,0,0
4,福岡,薬院・平尾・高宮,Lu’ce coco【ルーチェ ココ】,https://beauty.hotpepper.jp/slnH000284489/?cstt=1,福岡県福岡市南区大楠３-12-21モナーク高宮1F,西鉄高宮駅徒歩4分 インナーカラー/髪質改善/キッズカット/イルミナカラー/縮毛矯正,3850,8,1346,233,8.0,【透明感UP☆】カット＋イルミナカラー＋オージュアリペアTR¥14300→9350,9350,【透明感UP☆】カット＋イルミナカラー＋オージュアリペアTR￥14300→￥8800,8800,【透明感UP☆】カット＋イルミナカラー＋オージュアリペアTR¥14300→￥11000,11000,51,35,138,1346,4.60,4.6,4.7,4.6,4.4,ゆうきさん,2023/3/19,4,4,5,4,4,先日は口うるさい私の注文にお付き合い頂き、本当にありがとうございました!\n長年通っていた美...,カット+イルミナカラー+デジタルパーマ+オージュアリペアTR\26950→ \17600,カット、カラー、パーマ、縮毛矯正、トリートメント,無,8000,8999,8000,8999,女性,40代,その他,4,1,1,1,1,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17791,福岡,薬院・平尾・高宮,la paix【ラペ】,https://beauty.hotpepper.jp/slnH000050281/?cst...,福岡県福岡市中央区薬院１－１６－１８ 江島ビル３０５号,西鉄・地下鉄薬院駅 徒歩3分,4860,3,0,3,3.0,,0,,0,,0,0,35,0,0,5.00,5.0,5.0,5.0,4.0,みさきさん,2022/11/18,5,5,5,5,4,先日縮毛矯正を初めてかけてもらいました。\nとってもサラサラで髪の毛が生まれ変わったようで毎...,クーポン利用なし,縮毛矯正、トリートメント、その他,無,17000,17999,12000,12999,女性,30代前半,会社員,3,0,0,1,0,1,1,0,0,0,0,0
17792,福岡,薬院・平尾・高宮,la paix【ラペ】,https://beauty.hotpepper.jp/slnH000050281/?cst...,福岡県福岡市中央区薬院１－１６－１８ 江島ビル３０５号,西鉄・地下鉄薬院駅 徒歩3分,4860,3,0,3,3.0,,0,,0,,0,0,35,0,0,5.00,5.0,5.0,5.0,4.0,みさきさん,2022/4/23,5,5,5,5,5,去年から気になっててやっと伺うことが出来ました。\nLINEでのやり取りからすごく丁寧で安心...,クーポン利用なし,縮毛矯正、トリートメント、その他,無,17000,17999,12000,12999,女性,20代前半,会社員,3,0,0,1,0,1,1,0,0,0,0,0
17793,福岡,薬院・

In [69]:
df.dropna(subset=['投稿日時'])

,県,エリア,サロン名,URL,住所,アクセス,カット料金,セット面の数,ブログ投稿数,口コミ数,スタッフ数,1番人気クーポン名,1番人気クーポン価格,2番人気クーポン名,2番人気クーポン価格,3番人気クーポン名,3番人気クーポン価格,クーポン数,メニュー数,スタイル数,ブログ数,口コミスコアサロン全平均,雰囲気の全平均,接客サービスの全平均,技術・仕上がりの全平均,メニュー。料金の全平均,名前,投稿日時,総合,雰囲気,接客サービス,技術・仕上がり,メニュー・料金,口コミ本文,選択されたクーポン,メニューの種類,コメントへの返信,初回予約金額最小,初回予約金額最大,2回目以降予約金額最小,2回目以降予約金額最大,性別,年齢,職業,駅徒歩,カット選択,カラー選択,トリートメント選択,パーマ選択,縮毛矯正選択,その他選択,ヘッドスパ選択,ヘアセット選択,エクステ選択,着付け選択,メニュー無し選択
0,福岡,薬院・平尾・高宮,Lu’ce coco【ルーチェ ココ】,https://beauty.hotpepper.jp/slnH000284489/?cstt=1,福岡県福岡市南区大楠３-12-21モナーク高宮1F,西鉄高宮駅徒歩4分 インナーカラー/髪質改善/キッズカット/イルミナカラー/縮毛矯正,3850,8,1346,233,8.0,【透明感UP☆】カット＋イルミナカラー＋オージュアリペアTR¥14300→9350,9350,【透明感UP☆】カット＋イルミナカラー＋オージュアリペアTR￥14300→￥8800,8800,【透明感UP☆】カット＋イルミナカラー＋オージュアリペアTR¥14300→￥11000,11000,51,35,138,1346,4.60,4.6,4.7,4.6,4.4,siinaさん,2023/4/22,5,5,5,5,5,2回目の利用で、今回はイルミナカラーとカットをお願いしました！数年ぶりに前髪を作りたいと思い...,【透明感UP☆】カット＋イルミナカラー＋オージュアリペアTR\14300→\9900,カット、カラー,無,8000,8999,8000,8999,女性,30代前半,パート・アルバイト,4,1,1,0,0,0,0,0,0,0,0,0
1,福岡,薬院・平尾・高宮,Lu’ce coco【ルーチェ ココ】,https://beauty.hotpepper.jp/slnH000284489/?cstt=1,福岡県福岡市南区大楠３-12-21モナーク高宮1F,西鉄高宮駅徒歩4分 インナーカラー/髪質改善/キッズカット/イルミナカラー/縮毛矯正,3850,8,1346,233,8.0,【透明感UP☆】カット＋イルミナカラー＋オージュアリペアTR¥14300→9350,9350,【透明感UP☆】カット＋イルミナカラー＋オージュアリペアTR￥14300→￥8800,8800,【透明感UP☆】カット＋イルミナカラー＋オージュアリペアTR¥14300→￥11000,11000,51,35,138,1346,4.60,4.6,4.7,4.6,4.4,まなみんさん,2023/4/18,4,5,4,5,5,いろいろ説明してもらって説明の仕方もわかりやすかったので有り難かったです!!\n途中でカラー...,クーポン利用なし,カット、カラー,無,8000,8999,8000,8999,女性,30代後半,パート・アルバイト,4,1,1,0,0,0,0,0,0,0,0,0
2,福岡,薬院・平尾・高宮,Lu’ce coco【ルーチェ ココ】,https://beauty.hotpepper.jp/slnH000284489/?cstt=1,福岡県福岡市南区大楠３-12-21モナーク高宮1F,西鉄高宮駅徒歩4分 インナーカラー/髪質改善/キッズカット/イルミナカラー/縮毛矯正,3850,8,1346,233,8.0,【透明感UP☆】カット＋イルミナカラー＋オージュアリペアTR¥14300→9350,9350,【透明感UP☆】カット＋イルミナカラー＋オージュアリペアTR￥14300→￥8800,8800,【透明感UP☆】カット＋イルミナカラー＋オージュアリペアTR¥14300→￥11000,11000,51,35,138,1346,4.60,4.6,4.7,4.6,4.4,けんけんさん,2023/4/13,5,5,5,5,5,毎回いい感じにしていただいてありがとうございます,【大人気☆】メンズカット＋カラー＋シャンプー\12100→￥7700,カット、カラー,無,8000,8999,8000,8999,男性,20代後半,不明,4,1,1,0,0,0,0,0,0,0,0,0
3,福岡,薬院・平尾・高宮,Lu’ce coco【ルーチェ ココ】,https://beauty.hotpepper.jp/slnH000284489/?cstt=1,福岡県福岡市南区大楠３-12-21モナーク高宮1F,西鉄高宮駅徒歩4分 インナーカラー/髪質改善/キッズカット/イルミナカラー/縮毛矯正,3850,8,1346,233,8.0,【透明感UP☆】カット＋イルミナカラー＋オージュアリペアTR¥14300→9350,9350,【透明感UP☆】カット＋イルミナカラー＋オージュアリペアTR￥14300→￥8800,8800,【透明感UP☆】カット＋イルミナカラー＋オージュアリペアTR¥14300→￥11000,11000,51,35,138,1346,4.60,4.6,4.7,4.6,4.4,LGMさん,2023/4/4,2,3,3,2,2,キッズカットをお願いしたのですが後ろの真ん中長い野がってガタガタです…。立たせて切ってて娘も...,【ママさん限定】カット＋カラー＋TR & [キッズカット無料] \12100→￥8250,カット、カラー、トリートメント,無,8000,8999,8000,8999,女性,30代後半,主婦,4,1,1,1,0,0,0,0,0,0,0,0
4,福岡,薬院・平尾・高宮,Lu’ce coco【ルーチェ ココ】,https://beauty.hotpepper.jp/slnH000284489/?cstt=1,福岡県福岡市南区大楠３-12-21モナーク高宮1F,西鉄高宮駅徒歩4分 インナーカラー/髪質改善/キッズカット/イルミナカラー/縮毛矯正,3850,8,1346,233,8.0,【透明感UP☆】カット＋イルミナカラー＋オージュアリペアTR¥14300→9350,9350,【透明感UP☆】カット＋イルミナカラー＋オージュアリペアTR￥14300→￥8800,8800,【透明感UP☆】カット＋イルミナカラー＋オージュアリペアTR¥14300→￥11000,11000,51,35,138,1346,4.60,4.6,4.7,4.6,4.4,ゆうきさん,2023/3/19,4,4,5,4,4,先日は口うるさい私の注文にお付き合い頂き、本当にありがとうございました!\n長年通っていた美...,カット+イルミナカラー+デジタルパーマ+オージュアリペアTR\26950→ \17600,カット、カラー、パーマ、縮毛矯正、トリートメント,無,8000,8999,8000,8999,女性,40代,その他,4,1,1,1,1,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17791,福岡,薬院・平尾・高宮,la paix【ラペ】,https://beauty.hotpepper.jp/slnH000050281/?cst...,福岡県福岡市中央区薬院１－１６－１８ 江島ビル３０５号,西鉄・地下鉄薬院駅 徒歩3分,4860,3,0,3,3.0,,0,,0,,0,0,35,0,0,5.00,5.0,5.0,5.0,4.0,みさきさん,2022/11/18,5,5,5,5,4,先日縮毛矯正を初めてかけてもらいました。\nとってもサラサラで髪の毛が生まれ変わったようで毎...,クーポン利用なし,縮毛矯正、トリートメント、その他,無,17000,17999,12000,12999,女性,30代前半,会社員,3,0,0,1,0,1,1,0,0,0,0,0
17792,福岡,薬院・平尾・高宮,la paix【ラペ】,https://beauty.hotpepper.jp/slnH000050281/?cst...,福岡県福岡市中央区薬院１－１６－１８ 江島ビル３０５号,西鉄・地下鉄薬院駅 徒歩3分,4860,3,0,3,3.0,,0,,0,,0,0,35,0,0,5.00,5.0,5.0,5.0,4.0,みさきさん,2022/4/23,5,5,5,5,5,去年から気になっててやっと伺うことが出来ました。\nLINEでのやり取りからすごく丁寧で安心...,クーポン利用なし,縮毛矯正、トリートメント、その他,無,17000,17999,12000,12999,女性,20代前半,会社員,3,0,0,1,0,1,1,0,0,0,0,0
17793,福岡,薬院・

In [70]:
df = df.dropna(subset=['投稿日時']).reset_index(drop=True)

In [71]:
df['投稿日時'] = df['投稿日時'].apply(lambda x: datetime.datetime.strptime(x, '%Y/%m/%d'))

In [72]:
df

,県,エリア,サロン名,URL,住所,アクセス,カット料金,セット面の数,ブログ投稿数,口コミ数,スタッフ数,1番人気クーポン名,1番人気クーポン価格,2番人気クーポン名,2番人気クーポン価格,3番人気クーポン名,3番人気クーポン価格,クーポン数,メニュー数,スタイル数,ブログ数,口コミスコアサロン全平均,雰囲気の全平均,接客サービスの全平均,技術・仕上がりの全平均,メニュー。料金の全平均,名前,投稿日時,総合,雰囲気,接客サービス,技術・仕上がり,メニュー・料金,口コミ本文,選択されたクーポン,メニューの種類,コメントへの返信,初回予約金額最小,初回予約金額最大,2回目以降予約金額最小,2回目以降予約金額最大,性別,年齢,職業,駅徒歩,カット選択,カラー選択,トリートメント選択,パーマ選択,縮毛矯正選択,その他選択,ヘッドスパ選択,ヘアセット選択,エクステ選択,着付け選択,メニュー無し選択
0,福岡,薬院・平尾・高宮,Lu’ce coco【ルーチェ ココ】,https://beauty.hotpepper.jp/slnH000284489/?cstt=1,福岡県福岡市南区大楠３-12-21モナーク高宮1F,西鉄高宮駅徒歩4分 インナーカラー/髪質改善/キッズカット/イルミナカラー/縮毛矯正,3850,8,1346,233,8.0,【透明感UP☆】カット＋イルミナカラー＋オージュアリペアTR¥14300→9350,9350,【透明感UP☆】カット＋イルミナカラー＋オージュアリペアTR￥14300→￥8800,8800,【透明感UP☆】カット＋イルミナカラー＋オージュアリペアTR¥14300→￥11000,11000,51,35,138,1346,4.60,4.6,4.7,4.6,4.4,siinaさん,2023-04-22,5,5,5,5,5,2回目の利用で、今回はイルミナカラーとカットをお願いしました！数年ぶりに前髪を作りたいと思い...,【透明感UP☆】カット＋イルミナカラー＋オージュアリペアTR\14300→\9900,カット、カラー,無,8000,8999,8000,8999,女性,30代前半,パート・アルバイト,4,1,1,0,0,0,0,0,0,0,0,0
1,福岡,薬院・平尾・高宮,Lu’ce coco【ルーチェ ココ】,https://beauty.hotpepper.jp/slnH000284489/?cstt=1,福岡県福岡市南区大楠３-12-21モナーク高宮1F,西鉄高宮駅徒歩4分 インナーカラー/髪質改善/キッズカット/イルミナカラー/縮毛矯正,3850,8,1346,233,8.0,【透明感UP☆】カット＋イルミナカラー＋オージュアリペアTR¥14300→9350,9350,【透明感UP☆】カット＋イルミナカラー＋オージュアリペアTR￥14300→￥8800,8800,【透明感UP☆】カット＋イルミナカラー＋オージュアリペアTR¥14300→￥11000,11000,51,35,138,1346,4.60,4.6,4.7,4.6,4.4,まなみんさん,2023-04-18,4,5,4,5,5,いろいろ説明してもらって説明の仕方もわかりやすかったので有り難かったです!!\n途中でカラー...,クーポン利用なし,カット、カラー,無,8000,8999,8000,8999,女性,30代後半,パート・アルバイト,4,1,1,0,0,0,0,0,0,0,0,0
2,福岡,薬院・平尾・高宮,Lu’ce coco【ルーチェ ココ】,https://beauty.hotpepper.jp/slnH000284489/?cstt=1,福岡県福岡市南区大楠３-12-21モナーク高宮1F,西鉄高宮駅徒歩4分 インナーカラー/髪質改善/キッズカット/イルミナカラー/縮毛矯正,3850,8,1346,233,8.0,【透明感UP☆】カット＋イルミナカラー＋オージュアリペアTR¥14300→9350,9350,【透明感UP☆】カット＋イルミナカラー＋オージュアリペアTR￥14300→￥8800,8800,【透明感UP☆】カット＋イルミナカラー＋オージュアリペアTR¥14300→￥11000,11000,51,35,138,1346,4.60,4.6,4.7,4.6,4.4,けんけんさん,2023-04-13,5,5,5,5,5,毎回いい感じにしていただいてありがとうございます,【大人気☆】メンズカット＋カラー＋シャンプー\12100→￥7700,カット、カラー,無,8000,8999,8000,8999,男性,20代後半,不明,4,1,1,0,0,0,0,0,0,0,0,0
3,福岡,薬院・平尾・高宮,Lu’ce coco【ルーチェ ココ】,https://beauty.hotpepper.jp/slnH000284489/?cstt=1,福岡県福岡市南区大楠３-12-21モナーク高宮1F,西鉄高宮駅徒歩4分 インナーカラー/髪質改善/キッズカット/イルミナカラー/縮毛矯正,3850,8,1346,233,8.0,【透明感UP☆】カット＋イルミナカラー＋オージュアリペアTR¥14300→9350,9350,【透明感UP☆】カット＋イルミナカラー＋オージュアリペアTR￥14300→￥8800,8800,【透明感UP☆】カット＋イルミナカラー＋オージュアリペアTR¥14300→￥11000,11000,51,35,138,1346,4.60,4.6,4.7,4.6,4.4,LGMさん,2023-04-04,2,3,3,2,2,キッズカットをお願いしたのですが後ろの真ん中長い野がってガタガタです…。立たせて切ってて娘も...,【ママさん限定】カット＋カラー＋TR & [キッズカット無料] \12100→￥8250,カット、カラー、トリートメント,無,8000,8999,8000,8999,女性,30代後半,主婦,4,1,1,1,0,0,0,0,0,0,0,0
4,福岡,薬院・平尾・高宮,Lu’ce coco【ルーチェ ココ】,https://beauty.hotpepper.jp/slnH000284489/?cstt=1,福岡県福岡市南区大楠３-12-21モナーク高宮1F,西鉄高宮駅徒歩4分 インナーカラー/髪質改善/キッズカット/イルミナカラー/縮毛矯正,3850,8,1346,233,8.0,【透明感UP☆】カット＋イルミナカラー＋オージュアリペアTR¥14300→9350,9350,【透明感UP☆】カット＋イルミナカラー＋オージュアリペアTR￥14300→￥8800,8800,【透明感UP☆】カット＋イルミナカラー＋オージュアリペアTR¥14300→￥11000,11000,51,35,138,1346,4.60,4.6,4.7,4.6,4.4,ゆうきさん,2023-03-19,4,4,5,4,4,先日は口うるさい私の注文にお付き合い頂き、本当にありがとうございました!\n長年通っていた美...,カット+イルミナカラー+デジタルパーマ+オージュアリペアTR\26950→ \17600,カット、カラー、パーマ、縮毛矯正、トリートメント,無,8000,8999,8000,8999,女性,40代,その他,4,1,1,1,1,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17789,福岡,薬院・平尾・高宮,la paix【ラペ】,https://beauty.hotpepper.jp/slnH000050281/?cst...,福岡県福岡市中央区薬院１－１６－１８ 江島ビル３０５号,西鉄・地下鉄薬院駅 徒歩3分,4860,3,0,3,3.0,,0,,0,,0,0,35,0,0,5.00,5.0,5.0,5.0,4.0,みさきさん,2022-11-18,5,5,5,5,4,先日縮毛矯正を初めてかけてもらいました。\nとってもサラサラで髪の毛が生まれ変わったようで毎...,クーポン利用なし,縮毛矯正、トリートメント、その他,無,17000,17999,12000,12999,女性,30代前半,会社員,3,0,0,1,0,1,1,0,0,0,0,0
17790,福岡,薬院・平尾・高宮,la paix【ラペ】,https://beauty.hotpepper.jp/slnH000050281/?cst...,福岡県福岡市中央区薬院１－１６－１８ 江島ビル３０５号,西鉄・地下鉄薬院駅 徒歩3分,4860,3,0,3,3.0,,0,,0,,0,0,35,0,0,5.00,5.0,5.0,5.0,4.0,みさきさん,2022-04-23,5,5,5,5,5,去年から気になっててやっと伺うことが出来ました。\nLINEでのやり取りからすごく丁寧で安心...,クーポン利用なし,縮毛矯正、トリートメント、その他,無,17000,17999,12000,12999,女性,20代前半,会社員,3,0,0,1,0,1,1,0,0,0,0,0
17791

In [73]:
mean_of_staff = df.dropna(subset=['スタッフ数'])['スタッフ数'].mean()

In [74]:
df['スタッフ数'] = df['スタッフ数'].fillna(mean_of_staff)

In [75]:
df['スタッフ数'] = df['スタッフ数'].astype(int)

In [76]:
df

,県,エリア,サロン名,URL,住所,アクセス,カット料金,セット面の数,ブログ投稿数,口コミ数,スタッフ数,1番人気クーポン名,1番人気クーポン価格,2番人気クーポン名,2番人気クーポン価格,3番人気クーポン名,3番人気クーポン価格,クーポン数,メニュー数,スタイル数,ブログ数,口コミスコアサロン全平均,雰囲気の全平均,接客サービスの全平均,技術・仕上がりの全平均,メニュー。料金の全平均,名前,投稿日時,総合,雰囲気,接客サービス,技術・仕上がり,メニュー・料金,口コミ本文,選択されたクーポン,メニューの種類,コメントへの返信,初回予約金額最小,初回予約金額最大,2回目以降予約金額最小,2回目以降予約金額最大,性別,年齢,職業,駅徒歩,カット選択,カラー選択,トリートメント選択,パーマ選択,縮毛矯正選択,その他選択,ヘッドスパ選択,ヘアセット選択,エクステ選択,着付け選択,メニュー無し選択
0,福岡,薬院・平尾・高宮,Lu’ce coco【ルーチェ ココ】,https://beauty.hotpepper.jp/slnH000284489/?cstt=1,福岡県福岡市南区大楠３-12-21モナーク高宮1F,西鉄高宮駅徒歩4分 インナーカラー/髪質改善/キッズカット/イルミナカラー/縮毛矯正,3850,8,1346,233,8,【透明感UP☆】カット＋イルミナカラー＋オージュアリペアTR¥14300→9350,9350,【透明感UP☆】カット＋イルミナカラー＋オージュアリペアTR￥14300→￥8800,8800,【透明感UP☆】カット＋イルミナカラー＋オージュアリペアTR¥14300→￥11000,11000,51,35,138,1346,4.60,4.6,4.7,4.6,4.4,siinaさん,2023-04-22,5,5,5,5,5,2回目の利用で、今回はイルミナカラーとカットをお願いしました！数年ぶりに前髪を作りたいと思い...,【透明感UP☆】カット＋イルミナカラー＋オージュアリペアTR\14300→\9900,カット、カラー,無,8000,8999,8000,8999,女性,30代前半,パート・アルバイト,4,1,1,0,0,0,0,0,0,0,0,0
1,福岡,薬院・平尾・高宮,Lu’ce coco【ルーチェ ココ】,https://beauty.hotpepper.jp/slnH000284489/?cstt=1,福岡県福岡市南区大楠３-12-21モナーク高宮1F,西鉄高宮駅徒歩4分 インナーカラー/髪質改善/キッズカット/イルミナカラー/縮毛矯正,3850,8,1346,233,8,【透明感UP☆】カット＋イルミナカラー＋オージュアリペアTR¥14300→9350,9350,【透明感UP☆】カット＋イルミナカラー＋オージュアリペアTR￥14300→￥8800,8800,【透明感UP☆】カット＋イルミナカラー＋オージュアリペアTR¥14300→￥11000,11000,51,35,138,1346,4.60,4.6,4.7,4.6,4.4,まなみんさん,2023-04-18,4,5,4,5,5,いろいろ説明してもらって説明の仕方もわかりやすかったので有り難かったです!!\n途中でカラー...,クーポン利用なし,カット、カラー,無,8000,8999,8000,8999,女性,30代後半,パート・アルバイト,4,1,1,0,0,0,0,0,0,0,0,0
2,福岡,薬院・平尾・高宮,Lu’ce coco【ルーチェ ココ】,https://beauty.hotpepper.jp/slnH000284489/?cstt=1,福岡県福岡市南区大楠３-12-21モナーク高宮1F,西鉄高宮駅徒歩4分 インナーカラー/髪質改善/キッズカット/イルミナカラー/縮毛矯正,3850,8,1346,233,8,【透明感UP☆】カット＋イルミナカラー＋オージュアリペアTR¥14300→9350,9350,【透明感UP☆】カット＋イルミナカラー＋オージュアリペアTR￥14300→￥8800,8800,【透明感UP☆】カット＋イルミナカラー＋オージュアリペアTR¥14300→￥11000,11000,51,35,138,1346,4.60,4.6,4.7,4.6,4.4,けんけんさん,2023-04-13,5,5,5,5,5,毎回いい感じにしていただいてありがとうございます,【大人気☆】メンズカット＋カラー＋シャンプー\12100→￥7700,カット、カラー,無,8000,8999,8000,8999,男性,20代後半,不明,4,1,1,0,0,0,0,0,0,0,0,0
3,福岡,薬院・平尾・高宮,Lu’ce coco【ルーチェ ココ】,https://beauty.hotpepper.jp/slnH000284489/?cstt=1,福岡県福岡市南区大楠３-12-21モナーク高宮1F,西鉄高宮駅徒歩4分 インナーカラー/髪質改善/キッズカット/イルミナカラー/縮毛矯正,3850,8,1346,233,8,【透明感UP☆】カット＋イルミナカラー＋オージュアリペアTR¥14300→9350,9350,【透明感UP☆】カット＋イルミナカラー＋オージュアリペアTR￥14300→￥8800,8800,【透明感UP☆】カット＋イルミナカラー＋オージュアリペアTR¥14300→￥11000,11000,51,35,138,1346,4.60,4.6,4.7,4.6,4.4,LGMさん,2023-04-04,2,3,3,2,2,キッズカットをお願いしたのですが後ろの真ん中長い野がってガタガタです…。立たせて切ってて娘も...,【ママさん限定】カット＋カラー＋TR & [キッズカット無料] \12100→￥8250,カット、カラー、トリートメント,無,8000,8999,8000,8999,女性,30代後半,主婦,4,1,1,1,0,0,0,0,0,0,0,0
4,福岡,薬院・平尾・高宮,Lu’ce coco【ルーチェ ココ】,https://beauty.hotpepper.jp/slnH000284489/?cstt=1,福岡県福岡市南区大楠３-12-21モナーク高宮1F,西鉄高宮駅徒歩4分 インナーカラー/髪質改善/キッズカット/イルミナカラー/縮毛矯正,3850,8,1346,233,8,【透明感UP☆】カット＋イルミナカラー＋オージュアリペアTR¥14300→9350,9350,【透明感UP☆】カット＋イルミナカラー＋オージュアリペアTR￥14300→￥8800,8800,【透明感UP☆】カット＋イルミナカラー＋オージュアリペアTR¥14300→￥11000,11000,51,35,138,1346,4.60,4.6,4.7,4.6,4.4,ゆうきさん,2023-03-19,4,4,5,4,4,先日は口うるさい私の注文にお付き合い頂き、本当にありがとうございました!\n長年通っていた美...,カット+イルミナカラー+デジタルパーマ+オージュアリペアTR\26950→ \17600,カット、カラー、パーマ、縮毛矯正、トリートメント,無,8000,8999,8000,8999,女性,40代,その他,4,1,1,1,1,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17789,福岡,薬院・平尾・高宮,la paix【ラペ】,https://beauty.hotpepper.jp/slnH000050281/?cst...,福岡県福岡市中央区薬院１－１６－１８ 江島ビル３０５号,西鉄・地下鉄薬院駅 徒歩3分,4860,3,0,3,3,,0,,0,,0,0,35,0,0,5.00,5.0,5.0,5.0,4.0,みさきさん,2022-11-18,5,5,5,5,4,先日縮毛矯正を初めてかけてもらいました。\nとってもサラサラで髪の毛が生まれ変わったようで毎...,クーポン利用なし,縮毛矯正、トリートメント、その他,無,17000,17999,12000,12999,女性,30代前半,会社員,3,0,0,1,0,1,1,0,0,0,0,0
17790,福岡,薬院・平尾・高宮,la paix【ラペ】,https://beauty.hotpepper.jp/slnH000050281/?cst...,福岡県福岡市中央区薬院１－１６－１８ 江島ビル３０５号,西鉄・地下鉄薬院駅 徒歩3分,4860,3,0,3,3,,0,,0,,0,0,35,0,0,5.00,5.0,5.0,5.0,4.0,みさきさん,2022-04-23,5,5,5,5,5,去年から気になっててやっと伺うことが出来ました。\nLINEでのやり取りからすごく丁寧で安心...,クーポン利用なし,縮毛矯正、トリートメント、その他,無,17000,17999,12000,12999,女性,20代前半,会社員,3,0,0,1,0,1,1,0,0,0,0,0
17791,福岡,薬院・平尾・高宮,l

In [77]:
df['イルミナメニュー化の有無'] = 0

In [78]:
def illumina_use_judge(df, column):
    df.loc[df[column].str.contains('イルミナ'), 'イルミナメニュー化の有無'] = 1
    df.loc[df[column].str.contains('ILLUMINA'), 'イルミナメニュー化の有無'] = 1
    df.loc[df[column].str.contains('ｲﾙﾐﾅ'), 'イルミナメニュー化の有無'] = 1
    df.loc[df[column].str.contains('illumina'), 'イルミナメニュー化の有無'] = 1
    df.loc[df[column].str.contains('Illumina'), 'イルミナメニュー化の有無'] = 1
    df.loc[df[column].str.contains('ｉｌｌｕｍｉｎａ'), 'イルミナメニュー化の有無'] = 1
    df.loc[df[column].str.contains('Ｉｌｌｕｍｉｎａ'), 'イルミナメニュー化の有無'] = 1

In [79]:
illumina_use_judge(df=df, column='1番人気クーポン名')
illumina_use_judge(df=df, column='2番人気クーポン名')
illumina_use_judge(df=df, column='3番人気クーポン名')

In [80]:
df.to_csv('一時保存.csv', index=False)

In [81]:
for salon_name in df['サロン名']:
    if df[df['サロン名']==salon_name]['選択されたクーポン'].str.contains('イルミナ').sum():
        df.loc[df['サロン名']==salon_name, 'イルミナメニュー化の有無'] = 1
    if df[df['サロン名']==salon_name]['選択されたクーポン'].str.contains('ILLUMINA').sum():
        df.loc[df['サロン名']==salon_name, 'イルミナメニュー化の有無'] = 1
    if df[df['サロン名']==salon_name]['選択されたクーポン'].str.contains('ｲﾙﾐﾅ').sum():
        df.loc[df['サロン名']==salon_name, 'イルミナメニュー化の有無'] = 1
    if df[df['サロン名']==salon_name]['選択されたクーポン'].str.contains('illumina').sum():
        df.loc[df['サロン名']==salon_name, 'イルミナメニュー化の有無'] = 1
    if df[df['サロン名']==salon_name]['選択されたクーポン'].str.contains('Illumina').sum():
        df.loc[df['サロン名']==salon_name, 'イルミナメニュー化の有無'] = 1
    if df[df['サロン名']==salon_name]['選択されたクーポン'].str.contains('ｉｌｌｕｍｉｎａ').sum():
        df.loc[df['サロン名']==salon_name, 'イルミナメニュー化の有無'] = 1
    if df[df['サロン名']==salon_name]['選択されたクーポン'].str.contains('Ｉｌｌｕｍｉｎａ').sum():
        df.loc[df['サロン名']==salon_name, 'イルミナメニュー化の有無'] = 1    

KeyboardInterrupt: 

In [85]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17794 entries, 0 to 17793
Data columns (total 57 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   県             17794 non-null  object        
 1   エリア           17794 non-null  object        
 2   サロン名          17794 non-null  object        
 3   URL           17794 non-null  object        
 4   住所            17794 non-null  object        
 5   アクセス          17794 non-null  object        
 6   カット料金         17794 non-null  int64         
 7   セット面の数        17794 non-null  int64         
 8   ブログ投稿数        17794 non-null  int64         
 9   口コミ数          17794 non-null  int64         
 10  スタッフ数         17794 non-null  int64         
 11  1番人気クーポン名     17794 non-null  object        
 12  1番人気クーポン価格    17794 non-null  object        
 13  2番人気クーポン名     17794 non-null  object        
 14  2番人気クーポン価格    17794 non-null  object        
 15  3番人気クーポン名     17794 non-null  object

In [ ]:
df['Aujuaメニュー化の有無'] = 0

In [ ]:
def aujua_use_judge(df, column):
    df.loc[df[column].str.contains('オージュア'), 'Aujuaメニュー化の有無'] = 1
    df.loc[df[column].str.contains('ｵｰｼﾞｭｱ'), 'Aujuaメニュー化の有無'] = 1
    df.loc[df[column].str.contains('Aujua'), 'Aujuaメニュー化の有無'] = 1
    df.loc[df[column].str.contains('AUJUA'), 'Aujuaメニュー化の有無'] = 1
    df.loc[df[column].str.contains('aujua'), 'Aujuaメニュー化の有無'] = 1
    df.loc[df[column].str.contains('ａｕｊｕａ'), 'Aujuaメニュー化の有無'] = 1
    df.loc[df[column].str.contains('Ａｕｊｕａ'), 'Aujuaメニュー化の有無'] = 1

In [ ]:
aujua_use_judge(df=df, column='1番人気クーポン名')
aujua_use_judge(df=df, column='2番人気クーポン名')
aujua_use_judge(df=df, column='3番人気クーポン名')

In [ ]:
df.to_csv('一時保存.csv', index=False)

In [ ]:
for salon_name in df['サロン名']:
    if df[df['サロン名']==salon_name]['選択されたクーポン'].str.contains('オージュア').sum():
        df.loc[df['サロン名']==salon_name, 'Aujuaメニュー化の有無'] = 1
    if df[df['サロン名']==salon_name]['選択されたクーポン'].str.contains('ｵｰｼﾞｭｱ').sum():
        df.loc[df['サロン名']==salon_name, 'Aujuaメニュー化の有無'] = 1
    if df[df['サロン名']==salon_name]['選択されたクーポン'].str.contains('Aujua').sum():
        df.loc[df['サロン名']==salon_name, 'Aujuaメニュー化の有無'] = 1
    if df[df['サロン名']==salon_name]['選択されたクーポン'].str.contains('aujua').sum():
        df.loc[df['サロン名']==salon_name, 'Aujuaメニュー化の有無'] = 1
    if df[df['サロン名']==salon_name]['選択されたクーポン'].str.contains('ａｕｊｕａ').sum():
        df.loc[df['サロン名']==salon_name, 'Aujuaメニュー化の有無'] = 1
    if df[df['サロン名']==salon_name]['選択されたクーポン'].str.contains('Ａｕｊｕａ').sum():
        df.loc[df['サロン名']==salon_name, 'Aujuaメニュー化の有無'] = 1

In [ ]:
df.to_csv('一時保存.csv', index=False)

In [ ]:
df['住所_編集'] = df['住所'].apply(lambda x: mojimoji.zen_to_han(x))

In [ ]:
def get_lat_lon_from_address(address: str) -> dict:
    """
    address_lにlistの形で住所を入れてあげると、latlonsという入れ子上のリストで緯度経度のリストを返す関数。
    >>>>get_lat_lon_from_address(['東京都文京区本郷7-3-1','東京都文京区湯島３丁目３０−１'])
    [['35.712056', '139.762775'], ['35.707771', '139.768205']]
    """
    url = 'http://www.geocoding.jp/api/'
    payload = {"v": 1.1, 'q': address}
    r = requests.get(url, params=payload)
    ret = BeautifulSoup(r.content,'lxml')
    if ret.find('error'):
        raise ValueError(f"Invalid address submitted. {address}")
    else:
        lat = ret.find('lat').string
        lon = ret.find('lng').string
        location_dict = {'lat': lat, 'lon': lon}
        time.sleep(10) # sleep(10)は絶対必要
    return location_dict

In [ ]:
get_lat_lon_from_address('福岡県福岡市南区大楠３-12-21モナーク高宮1F')

/Users/masudaniwabinari/Desktop/hpb_scraping/.env/lib/python3.8/site-packages/bs4/builder/__init__.py:545: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.
  warnings.warn(


{'lat': '33.57004', 'lon': '130.412914'}

In [ ]:
for address in df['住所_編集'].unique():
    print(address)
    location_dict = get_lat_lon_from_address(address)
    print(location_dict)
    print(location_dict['lat'])
    print(location_dict['lon'])
    
    df.loc[df['住所_編集']]
    break

福岡県福岡市南区大楠3-12-21ﾓﾅｰｸ高宮1F


/Users/masudaniwabinari/Desktop/hpb_scraping/.env/lib/python3.8/site-packages/bs4/builder/__init__.py:545: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.
  warnings.warn(


{'lat': '33.57004', 'lon': '130.412914'}
33.57004
130.412914


In [ ]:
i=0
for _list in df['選択されたクーポン'].str.split('\\'):
    print(_list[-1])
    i+=1
    if i==9:
        break

9900
クーポン利用なし
12100→￥7700
12100→￥8250
17600
クーポン利用なし
【限定クーポン】カット＋マイクロバブル￥6050→￥4950
19250→￥14850
12100→￥7700


In [ ]:
_list=[]
for row in df['選択されたクーポン']:
    if '\\' not in row:
        if '¥' not in row:
            if '￥' not in row:
                _list.append(row)
print(set(_list))

{"☆リニューアル記念☆【men's限定】ブリーチ＋カラー＋カット", '★メンテナンスをしたい方★カット（シャンプー無し・要詳細）/3000', '【人気no.2/白髪染変更OK】カット+根元カラーリタッチ\u300011,550円→6,000円', '【潤いUP】カット+オーガニックカラー+トリートメント\u300011550円→8500円', '『朝も楽ちん♪』カット＋パーマ＋潤いオーガニックトリートメント', '【新規】カット+カラー+しっとりトリートメント13200\u3000→9240', '【髪質改善☆高濃度補修でうるツヤ美髪】カット+カラー+プリフィカTr\u30009400', '【ミルボン4STEPトリ付】カット+カラー\u300019,910円→8,700円', '【山中指名限定】フルコース髪質改善オージュア＋メリートリートメント', '【新規限定☆】小顔カット＋抜け感カラー＋トリートメント12400円☆', '【最高な艶髪に♪】カット+ケアカラー+4step超音波トリートメント', '美髪ストレート+カット＋極艶TR+マイナスTR', 'カット＋ダメージレスカラー＋艶髪ラメラメトリートメント', '◇メンズ限定◇似合わせカット+カラー', 'カットのクーポンはこちら↓', '【メンズ限定★人気No,1】\u3000カット＋極上のヘッドスパ\u30008800円→4900円', '【男女共通選べるパーマ】カット+パーマ', '前回と同じメニュ・', '【再来クーポン/キマる髪型♪】メンズカット\u30005,280円→3,000円', '初めての方はこちら【脱白髪染め】ハイライト＋カット14130～17980※詳細確認', 'NEW☆【キャンペーン】カット+オーガニックカラー+トリートメント\u30007000円', '【至高のケア・リピートNo.1】カット+カラー(白髪染め可)+TOKIO TR 14300円', '【ダメージレス×デザイン】カット＋白髪ぼかしハイライト', '艶髪カラー+カット＋マイナスTR+極艶TR+【エイジングスパ付】', '【お得!平日限定】カット+grade1トリートメント4950円', '【学割U24】中学・高校生カットコース＋選べるトリートメント付き', '【ご来店3回目まで限定】カラー+パーマ+カット+トリート

In [ ]:
df['選択されたクーポン_編集'] = df['選択されたクーポン'].apply(lambda x: mojimoji.zen_to_han(x, kana=False))

In [ ]:
df['選択されたクーポン_編集'].str.contains('¥').sum()

3196

In [ ]:
df.to_csv('一時保存.csv', index=False)

In [ ]:
df['選択されたクーポン_編集'] = df['選択されたクーポン_編集'].str.replace('　', '').str.replace(' ', '')

In [ ]:
# ひらがな消す。
p = re.compile('[\u3041-\u309F]+')

In [ ]:
df['選択されたクーポン_編集'] = df['選択されたクーポン_編集'].apply(lambda x: p.sub('', x))

In [ ]:
df.to_csv('一時保存.csv', index=False)

In [ ]:
# 次にカタカナを消す。
p = re.compile('[\u30A1-\u30FF]+')

In [ ]:
df['選択されたクーポン_編集'] = df['選択されたクーポン_編集'].apply(lambda x: p.sub('', x))

In [ ]:
df.to_csv('一時保存.csv', index=False)

In [ ]:
!pip install regex
import regex

You should consider upgrading via the '/Users/masudaniwabinari/Desktop/hpb_scraping/.env/bin/python -m pip install --upgrade pip' command.


In [ ]:
p = regex.compile(r'\p{Script_Extensions=Han}+')

In [ ]:
df['選択されたクーポン_編集'] = df['選択されたクーポン_編集'].apply(lambda x: p.sub('', x))

In [ ]:
df[['選択されたクーポン_編集']].to_csv('一時保存_クーポンのみ.csv', index=False)

In [ ]:
df['選択されたクーポン_編集'] = df['選択されたクーポン_編集'].str.replace(',', '').str.replace('，', '')

In [ ]:
df[['選択されたクーポン_編集']].to_csv('一時保存_クーポンのみ.csv', index=False)

In [ ]:
p = re.compile('[a-zA-Z]+')
df['選択されたクーポン_編集'] = df['選択されたクーポン_編集'].apply(lambda x: p.sub('', x))
df[['選択されたクーポン_編集']].to_csv('一時保存_クーポンのみ.csv', index=False)

In [ ]:
df['選択されたクーポン_編集'] = df['選択されたクーポン_編集'].apply(lambda x: x[-5:])

In [ ]:
df[['選択されたクーポン_編集']].to_csv('一時保存_クーポンのみ.csv', index=False)

In [ ]:
p = re.compile('[^0-9]+')
df['選択されたクーポン_編集'] = df['選択されたクーポン_編集'].apply(lambda x: p.sub('', x))
df[['選択されたクーポン_編集']].to_csv('一時保存_クーポンのみ.csv', index=False)

In [ ]:
df['選択されたクーポン_編集'].apply(lambda x: len(x))

0        4
1        0
2        4
3        4
4        5
        ..
17789    0
17790    0
17791    0
17792    0
17793    0
Name: 選択されたクーポン_編集, Length: 17794, dtype: int64

In [ ]:
def delete_not_payment_data(x):
    if len(x) < 4:
        return pd.NA
    else:
        return int(x)

In [ ]:
df['選択されたクーポン_編集'] = df['選択されたクーポン_編集'].apply(lambda x: delete_not_payment_data(x))

In [ ]:
df['選択されたクーポン_編集'] = df['選択されたクーポン_編集'].astype(pd.Int64Dtype())

In [ ]:
df['選択されたクーポン_編集']

0         9900
1         <NA>
2         7700
3         8250
4        17600
         ...  
17789     <NA>
17790     <NA>
17791     <NA>
17792     <NA>
17793     <NA>
Name: 選択されたクーポン_編集, Length: 17794, dtype: Int64

In [ ]:
df[['選択されたクーポン_編集']].to_csv('一時保存_クーポンのみ.csv', index=False)

In [ ]:
df.rename(columns={'選択されたクーポン_編集':'支出金額'}, inplace=True)

In [ ]:
df.to_csv('一時保存.csv', index=False)

In [ ]:
df['セット面の数']

NameError: name 'df' is not defined

In [86]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17794 entries, 0 to 17793
Data columns (total 57 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   県             17794 non-null  object        
 1   エリア           17794 non-null  object        
 2   サロン名          17794 non-null  object        
 3   URL           17794 non-null  object        
 4   住所            17794 non-null  object        
 5   アクセス          17794 non-null  object        
 6   カット料金         17794 non-null  int64         
 7   セット面の数        17794 non-null  int64         
 8   ブログ投稿数        17794 non-null  int64         
 9   口コミ数          17794 non-null  int64         
 10  スタッフ数         17794 non-null  int64         
 11  1番人気クーポン名     17794 non-null  object        
 12  1番人気クーポン価格    17794 non-null  object        
 13  2番人気クーポン名     17794 non-null  object        
 14  2番人気クーポン価格    17794 non-null  object        
 15  3番人気クーポン名     17794 non-null  object